In [1]:
# Parameters
run_date = "2026-02-04"
output_dir = "C:\\datum-api-examples-main\\OriON\\signals"


In [2]:
# Parameters
run_date = "2026-01-01"  # papermill replacement
import os
output_dir = os.environ.get("ORION_SIGNALS_DIR", "../signals")
config_path = os.environ.get("DATUM_API_CONFIG_PATH", "../ops/datum_api_config.json")
dry_run = False

# ensure output exists
os.makedirs(output_dir, exist_ok=True)


In [3]:
# Import basic modules
import pandas as pd
from datum_api_client import DatumApi
import datetime
from datetime import timedelta
import gzip
from pathlib import Path


# Import warnings
import warnings
warnings.filterwarnings("ignore")
# pip install xlrd
# pip install openpyxl
import os
from pathlib import Path

In [4]:
from __future__ import annotations


def devsig_stream_stats_v12_exporter(
    input_path: str,
    *,
    # === outputs ===
    output_onefile_jsonl: str = "ARBITRAGE/onefile.jsonl",
    output_summary_csv: str = "ARBITRAGE/summary.csv",
    output_best_params_jsonl: str = "ARBITRAGE/best_params.jsonl",
    # === include heavy parts ===
    include_events_pre: bool = False,
    include_events_intra: bool = False,
    include_events_post: bool = False,
    max_events_per_ticker: int = 500,
    # === thresholds ===
    dev_thr: float = 0.30,      # trigger (abs(dev_sig) >= dev_thr)
    norm_thr: float = 0.10,     # HARD normalization threshold (abs(dev_sig) <= norm_thr)
    soft_ratio: float = 3.0,    # SOFT: abs(dev_sig) <= peak_abs / soft_ratio
    # === best params selection rules (kept; we ADD simpler ANY windows) ===
    best_rules: "dict|None" = None,
    # === reading ===
    assume_sorted: bool = True,
    parquet_use_pyarrow: bool = True,
    parquet_iter_batches: bool = True,   # ✅ optional speedup (Step 4)
    parquet_batch_size: int = 1_000_000, # ✅ batch size for iter_batches
    csv_chunksize: int = 500_000,
    log_every_n_chunks: int = 5,
    # === bins ===
    sigma_bin_min: float = 0.2,
    sigma_bin_max: float = 2.7,
    sigma_bin_step: float = 0.1,
    bench_bin_min: float = -3.0,
    bench_bin_max: float = 3.0,
    bench_bin_step: float = 0.1,
    # === time bands ===
    start_band_minutes: int = 30,
    norm_band_minutes: int = 30,
    # === numeric fields stored in data ===
    BENCH_NUM_FIELD: str = "Bench%",
    STOCK_NUM_FIELD: str = "Stack%",
    # === global filter for ALL outputs ===
    min_events_per_ticker: int = 10,
    # === open series ===
    open_series_downsample_seconds: int = 60,  # 60s => 1 point / minute
):
    """
    v12 exporter UPDATED with BLUE + POST and strict "parallel class checks" semantics:

    ✅ Classes:
      - PRE classes: BLUE, ARK, PRINT, OPEN (all evaluated in parallel for the same PRE event)
      - GLOBAL = priority selector over {BLUE, ARK, PRINT, OPEN} (POST NOT included)
      - INTRA class (10:00–12:00)
      - POST class (16:01–19:59) (separate event stream, not in GLOBAL)

    ✅ GLOBAL priority:
      BLUE_HARD > ARK_HARD > PRINT_HARD > BLUE_SOFT > ARK_SOFT > PRINT_SOFT > OPEN_HARD > OPEN_SOFT > NONE

    ✅ BEST_PARAMS:
      - best_windows_any stitched for ALL classes:
        blue/ark/print/open/global/intra/post, each per sign (pos/neg)
      - uses ANY = hard+soft normalization ratio, thresholds total>=4, rate>=0.6

    ✅ IMPORTANT SEMANTICS:
      - BLUE/ARK/PRINT/OPEN do NOT mute each other. They all get their own hard/soft/none outcome.
      - PRE event is finalized after OPEN window (same as before).
      - BLUE has its OWN peak (frozen until 03:59) and soft is evaluated vs BLUE peak.
      - ARK/PRINT/OPEN use PRE peak frozen until 09:29.

    ✅ PERFORMANCE PATCHES (no semantic changes):
      - gzip outputs: .jsonl -> .jsonl.gz (if path endswith .gz)
      - vectorized dt/dev parsing + ignored window filtering before loop
      - avoid parse_dt()/hhmm()/is_ignored_time() per row
      - reduce gc.collect() frequency
      - optional pyarrow iter_batches() for parquet
    """
    import os, gc, json, time, math, gzip
    from collections import deque, defaultdict, Counter
    from datetime import datetime, timedelta
    import numpy as np
    import pandas as pd

    # ---------------- defaults for best rules (kept) ----------------
    if best_rules is None:
        best_rules = {
            "sigma_any":  {"min_rate": 0.60, "min_total": 20, "top_n": 3},
            "sigma_hard": {"min_rate": 0.55, "min_total": 20, "top_n": 3},
            "sigma_soft": {"min_rate": 0.60, "min_total": 15, "top_n": 3},

            "bench_any":  {"min_rate": 0.58, "min_total": 25, "top_n": 3},
            "bench_hard": {"min_rate": 0.52, "min_total": 25, "top_n": 3},
            "bench_soft": {"min_rate": 0.60, "min_total": 20, "top_n": 3},

            "soft_peak_rate": {"min_rate": 0.55, "min_total": 15, "top_n": 3},
            "soft_low_rate":  {"min_rate": 0.55, "min_total": 15, "top_n": 3},

            "start_band_any": {"min_rate": 0.60, "min_total": 20, "top_n": 3},
        }

    # ---------------- gzip-aware open ----------------
    def _open_text(path: str, mode: str = "wt"):
        # mode expected: "wt" or "at"
        if str(path).lower().endswith(".gz"):
            return gzip.open(
                path,
                mode,
                encoding="utf-8",
                newline="\n",
                compresslevel=6,  # баланс швидкість/розмір
            )
        return open(path, mode.replace("t", ""), encoding="utf-8", newline="\n")

    Path(output_onefile_jsonl).parent.mkdir(parents=True, exist_ok=True)
    Path(output_summary_csv).parent.mkdir(parents=True, exist_ok=True)
    Path(output_best_params_jsonl).parent.mkdir(parents=True, exist_ok=True)


    onefile_f = _open_text(output_onefile_jsonl, "wt")

    summary_cols = [
        "ticker", "bench", "events_total",
        "events_pre_total", "events_intra_total", "events_post_total",

        "blue_any_rate", "blue_hard_rate", "blue_soft_rate",
        "ark_any_rate", "ark_hard_rate", "ark_soft_rate",
        "print_any_rate", "print_hard_rate", "print_soft_rate",
        "open_any_rate", "open_hard_rate", "open_soft_rate",
        "global_any_rate", "global_hard_rate", "global_soft_rate",
        "intra_any_rate", "intra_hard_rate", "intra_soft_rate",
        "post_any_rate", "post_hard_rate", "post_soft_rate",

        "corr", "beta", "sigma",
    ]
    pd.DataFrame(columns=summary_cols).to_csv(output_summary_csv, index=False, mode="w")

    best_params_f = _open_text(output_best_params_jsonl, "wt")
    best_params_f.write(json.dumps({
        "meta": {"version": "v12+blue+post", "generated_at": datetime.utcnow().isoformat() + "Z"}
    }, ensure_ascii=False) + "\n")

    # ---------------- helpers ----------------
    def _json_safe(x):
        if x is None:
            return None
        if isinstance(x, (np.floating, float)):
            if np.isnan(x) or np.isinf(x):
                return None
            return float(x)
        if isinstance(x, (np.integer, int)):
            return int(x)
        if isinstance(x, (np.bool_, bool)):
            return bool(x)
        if isinstance(x, (pd.Timestamp,)):
            return x.isoformat()
        if isinstance(x, (datetime,)):
            return x.isoformat()
        return x

    def is_finite_num(x) -> bool:
        try:
            return np.isfinite(float(x))
        except Exception:
            return False

    def as_float_or_nan(x) -> float:
        try:
            return float(x)
        except Exception:
            return np.nan

    def hhmm(dt_obj):
        return (dt_obj.hour, dt_obj.minute) if isinstance(dt_obj, datetime) else None

    def in_range(t, a, b):
        return (t is not None) and (a <= t <= b)

    def _dt_iso(x):
        return x.isoformat() if isinstance(x, datetime) else None

    def floor_to_band(dt_obj: datetime, minutes: int) -> str:
        if not isinstance(dt_obj, datetime):
            return None
        m = (dt_obj.minute // minutes) * minutes
        start = dt_obj.replace(minute=m, second=0, microsecond=0)
        end = start + timedelta(minutes=minutes)
        return f"{start.hour:02d}:{start.minute:02d}-{end.hour:02d}:{end.minute:02d}"

    # ---------------- time windows ----------------
    BLUE_FROM = (0, 1)
    BLUE_TO   = (3, 59)

    ARK_FROM = (0, 5)
    ARK_TO   = (9, 29)

    PRINT_FROM = (9, 30)
    PRINT_TO   = (9, 35)

    OPEN_FROM  = (9, 31)
    OPEN_TO    = (9, 40)

    INTRA_FROM = (10, 0)
    INTRA_TO   = (12, 0)

    POST_FROM  = (16, 1)
    POST_TO    = (19, 59)

    # ✅ ignored windows (vectorized filtering uses these exact bounds, inclusive)
    IGNORE_WINDOWS = [((3, 58), (4, 5)), ((7, 58), (8, 5))]

    def is_ignored_time(t):
        return any(in_range(t, a, b) for a, b in IGNORE_WINDOWS)

    # ---------------- binning ----------------
    def _clamp(v, lo, hi):
        return max(lo, min(hi, v))

    def sigma_bin(abs_sigma):
        if not is_finite_num(abs_sigma):
            return None
        v = float(abs_sigma)
        v = _clamp(v, sigma_bin_min, sigma_bin_max)
        b = round(np.floor(v / sigma_bin_step) * sigma_bin_step, 1)
        return f"{b:.1f}"

    def bench_bin(val):
        if not is_finite_num(val):
            return None
        v = float(val)
        v = _clamp(v, bench_bin_min, bench_bin_max)
        b = round(np.floor(v / bench_bin_step) * bench_bin_step, 1)
        return f"{b:.1f}"

    def _score(rate: float, total: int) -> float:
        return float(rate) * math.log1p(int(total))

    # ---- “simple ANY windows” selection (rate>=0.6 & total>=4) ----
    def stitch_numeric_bin_intervals_from_any(
        bin_counts: dict, *, step: float, min_total: int = 4, min_rate: float = 0.6
    ):
        eligible = []
        for b_str, st in (bin_counts or {}).items():
            try:
                b = float(b_str)
            except Exception:
                continue
            total = int(st.get("total", 0))
            if total < min_total:
                continue
            any_ = int(st.get("hard", 0)) + int(st.get("soft", 0))
            rate = any_ / total if total else 0.0
            if rate >= min_rate:
                eligible.append(b)

        eligible.sort()
        if not eligible:
            return []

        intervals = []
        lo = hi = eligible[0]
        for b in eligible[1:]:
            if abs(b - (hi + step)) <= 1e-9:
                hi = b
            else:
                intervals.append((lo, hi))
                lo = hi = b
        intervals.append((lo, hi))

        out = []
        for lo, hi in intervals:
            tot = hard = soft = none = 0
            k = lo
            while k <= hi + 1e-9:
                ks = f"{k:.1f}"
                st = (bin_counts or {}).get(ks)
                if st:
                    tot += int(st.get("total", 0))
                    hard += int(st.get("hard", 0))
                    soft += int(st.get("soft", 0))
                    none += int(st.get("none", 0))
                k = round(k + step, 10)
            if tot <= 0:
                continue
            rate = (hard + soft) / tot
            if tot >= min_total and rate >= min_rate:
                out.append({
                    "lo": round(lo, 2),
                    "hi": round(hi, 2),
                    "total": int(tot),
                    "hard": int(hard),
                    "soft": int(soft),
                    "none": int(none),
                    "rate": float(rate),
                    "score": _score(rate, tot),
                })

        out.sort(key=lambda x: (x["score"], x["total"]), reverse=True)
        return out

    def stitch_timeband_intervals_from_any(
        band_counts: dict, *, min_total: int = 4, min_rate: float = 0.6
    ):
        def band_key_to_minutes(k: str):
            try:
                a, _b = k.split("-")
                h1, m1 = map(int, a.split(":"))
                return h1 * 60 + m1
            except Exception:
                return None

        items = []
        for k, st in (band_counts or {}).items():
            tot = int(st.get("total", 0))
            if tot < min_total:
                continue
            any_ = int(st.get("hard", 0)) + int(st.get("soft", 0))
            rate = any_ / tot if tot else 0.0
            if rate >= min_rate:
                km = band_key_to_minutes(k)
                if km is not None:
                    items.append((km, k))
        items.sort()
        if not items:
            return []

        stitched_groups = []
        cur = [items[0][1]]
        for _, k in items[1:]:
            prev = cur[-1]
            try:
                prev_end = prev.split("-")[1]
                k_start = k.split("-")[0]
                if prev_end == k_start:
                    cur.append(k)
                else:
                    stitched_groups.append(cur)
                    cur = [k]
            except Exception:
                stitched_groups.append(cur)
                cur = [k]
        stitched_groups.append(cur)

        stitched = []
        for bands in stitched_groups:
            tot = hard = soft = none = 0
            for b in bands:
                st = (band_counts or {}).get(b, {})
                tot += int(st.get("total", 0))
                hard += int(st.get("hard", 0))
                soft += int(st.get("soft", 0))
                none += int(st.get("none", 0))
            if tot <= 0:
                continue
            rate = (hard + soft) / tot
            if tot >= min_total and rate >= min_rate:
                stitched.append({
                    "from": bands[0],
                    "to": bands[-1],
                    "bands": bands,
                    "total": int(tot),
                    "hard": int(hard),
                    "soft": int(soft),
                    "none": int(none),
                    "rate": float(rate),
                    "score": _score(rate, tot),
                })
        stitched.sort(key=lambda x: (x["score"], x["total"]), reverse=True)
        return stitched

    # ---------------- global label priority (GLOBAL only) ----------------
    GLOBAL_PRIORITY = [
        "BLUE_HARD",
        "ARK_HARD",
        "PRINT_HARD",
        "BLUE_SOFT",
        "ARK_SOFT",
        "PRINT_SOFT",
        "OPEN_HARD",
        "OPEN_SOFT",
        "NONE",
    ]

    def compute_global_label(blue_status, ark_status, print_status, open_status):
        if blue_status == "hard":
            return "BLUE_HARD"
        if ark_status == "hard":
            return "ARK_HARD"
        if print_status == "hard":
            return "PRINT_HARD"
        if blue_status == "soft":
            return "BLUE_SOFT"
        if ark_status == "soft":
            return "ARK_SOFT"
        if print_status == "soft":
            return "PRINT_SOFT"
        if open_status == "hard":
            return "OPEN_HARD"
        if open_status == "soft":
            return "OPEN_SOFT"
        return "NONE"

    # ---------------- per-ticker state ----------------
    cur_ticker = None
    cur_day = None

    bench_name_seen = None
    static_triplet_set = False
    corr_static = beta_static = sigma_static = None

    # optional heavy buffers
    pre_events_buf = deque(maxlen=max_events_per_ticker)
    intra_events_buf = deque(maxlen=max_events_per_ticker)
    post_events_buf = deque(maxlen=max_events_per_ticker)

    # counts per class
    counts_pre = {
        "blue": Counter(),
        "ark": Counter(),
        "print": Counter(),
        "open": Counter(),
        "global": Counter(),
    }
    global_labels_counter = Counter()
    counts_intra = {"intra": Counter()}
    counts_post = {"post": Counter()}

    # sigma bins: class -> sign -> bin -> Counter(total/hard/soft/none)
    def make_sigma_bins_map(classes):
        m = {}
        for c in classes:
            m[c] = {"pos": defaultdict(lambda: Counter()), "neg": defaultdict(lambda: Counter())}
        return m

    sigma_bins_pre = make_sigma_bins_map(["blue", "ark", "print", "open", "global"])
    sigma_bins_intra = make_sigma_bins_map(["intra"])
    sigma_bins_post = make_sigma_bins_map(["post"])

    # bench bins
    def make_bench_bins_map(classes):
        out = {}
        for c in classes:
            out[c] = {
                "start": {"pos": defaultdict(lambda: Counter()), "neg": defaultdict(lambda: Counter())},
                "peak":  {"pos": defaultdict(lambda: Counter()), "neg": defaultdict(lambda: Counter())},
                "norm":  {"pos": defaultdict(lambda: Counter()), "neg": defaultdict(lambda: Counter())},
            }
        return out

    bench_bins_pre = make_bench_bins_map(["blue", "ark", "print", "open", "global"])
    bench_bins_intra = make_bench_bins_map(["intra"])
    bench_bins_post = make_bench_bins_map(["post"])

    # time bands (OLD ones kept) — PRE/INTRA/POST each
    start_bands_pre_total = Counter()
    start_bands_pre_any   = Counter()
    start_bands_pre_hard  = Counter()
    start_bands_pre_soft  = Counter()
    norm_bands_pre_any   = Counter()
    norm_bands_pre_hard  = Counter()
    norm_bands_pre_soft  = Counter()

    start_bands_intra_total = Counter()
    start_bands_intra_any   = Counter()
    start_bands_intra_hard  = Counter()
    start_bands_intra_soft  = Counter()
    norm_bands_intra_any   = Counter()
    norm_bands_intra_hard  = Counter()
    norm_bands_intra_soft  = Counter()

    start_bands_post_total = Counter()
    start_bands_post_any   = Counter()
    start_bands_post_hard  = Counter()
    start_bands_post_soft  = Counter()
    norm_bands_post_any   = Counter()
    norm_bands_post_hard  = Counter()
    norm_bands_post_soft  = Counter()

    # NEW: time bands per class+sign with total/hard/soft/none
    def make_timeband_map(classes):
        out = {}
        for c in classes:
            out[c] = {
                "start": {"pos": defaultdict(lambda: Counter()), "neg": defaultdict(lambda: Counter())},
                "norm":  {"pos": defaultdict(lambda: Counter()), "neg": defaultdict(lambda: Counter())},
            }
        return out

    timebands_pre_by_class_sign = make_timeband_map(["blue", "ark", "print", "open", "global"])
    timebands_intra_by_class_sign = make_timeband_map(["intra"])
    timebands_post_by_class_sign = make_timeband_map(["post"])

    # ✅ last3 examples per class AND per sign
    def make_last3_map(classes):
        return {c: {"pos": deque(maxlen=3), "neg": deque(maxlen=3)} for c in classes}

    last3_examples = make_last3_map(["blue", "ark", "print", "open", "global", "intra", "post"])

    # RECENT by DAYS (kept)
    recent_days = deque(maxlen=10)  # day strings YYYY-MM-DD
    recent_by_day = {}  # day -> {"print":..., "peak":...}

    last5_print_days_pos = deque(maxlen=5)
    last5_print_days_neg = deque(maxlen=5)

    last5_peak_days_pos = deque(maxlen=5)
    last5_peak_days_neg = deque(maxlen=5)

    # HARD delays (kept + blue/post)
    hard_delay_sum = Counter()
    hard_delay_cnt = Counter()

    # mean peak_abs for globally normalized events (kept) — now picks BLUE peak if global is BLUE
    global_norm_peak_sum = {"pos": 0.0, "neg": 0.0}
    global_norm_peak_cnt = {"pos": 0, "neg": 0}

    # OPEN dev_sig series for last10 days (downsample by seconds)
    open_series_by_day = {}

    # ---------------- reset ticker ----------------
    def reset_ticker_state():
        nonlocal bench_name_seen, static_triplet_set, corr_static, beta_static, sigma_static
        nonlocal pre_events_buf, intra_events_buf, post_events_buf
        nonlocal counts_pre, global_labels_counter, counts_intra, counts_post
        nonlocal sigma_bins_pre, sigma_bins_intra, sigma_bins_post
        nonlocal bench_bins_pre, bench_bins_intra, bench_bins_post
        nonlocal start_bands_pre_total, start_bands_pre_any, start_bands_pre_hard, start_bands_pre_soft
        nonlocal norm_bands_pre_any, norm_bands_pre_hard, norm_bands_pre_soft
        nonlocal start_bands_intra_total, start_bands_intra_any, start_bands_intra_hard, start_bands_intra_soft
        nonlocal norm_bands_intra_any, norm_bands_intra_hard, norm_bands_intra_soft
        nonlocal start_bands_post_total, start_bands_post_any, start_bands_post_hard, start_bands_post_soft
        nonlocal norm_bands_post_any, norm_bands_post_hard, norm_bands_post_soft
        nonlocal timebands_pre_by_class_sign, timebands_intra_by_class_sign, timebands_post_by_class_sign
        nonlocal last3_examples
        nonlocal recent_days, recent_by_day, last5_print_days_pos, last5_print_days_neg
        nonlocal last5_peak_days_pos, last5_peak_days_neg
        nonlocal hard_delay_sum, hard_delay_cnt
        nonlocal global_norm_peak_sum, global_norm_peak_cnt
        nonlocal open_series_by_day

        bench_name_seen = None
        static_triplet_set = False
        corr_static = beta_static = sigma_static = None

        pre_events_buf = deque(maxlen=max_events_per_ticker)
        intra_events_buf = deque(maxlen=max_events_per_ticker)
        post_events_buf = deque(maxlen=max_events_per_ticker)

        counts_pre = {"blue": Counter(), "ark": Counter(), "print": Counter(), "open": Counter(), "global": Counter()}
        global_labels_counter = Counter()
        counts_intra = {"intra": Counter()}
        counts_post = {"post": Counter()}

        sigma_bins_pre = make_sigma_bins_map(["blue", "ark", "print", "open", "global"])
        sigma_bins_intra = make_sigma_bins_map(["intra"])
        sigma_bins_post = make_sigma_bins_map(["post"])

        bench_bins_pre = make_bench_bins_map(["blue", "ark", "print", "open", "global"])
        bench_bins_intra = make_bench_bins_map(["intra"])
        bench_bins_post = make_bench_bins_map(["post"])

        start_bands_pre_total = Counter()
        start_bands_pre_any   = Counter()
        start_bands_pre_hard  = Counter()
        start_bands_pre_soft  = Counter()
        norm_bands_pre_any   = Counter()
        norm_bands_pre_hard  = Counter()
        norm_bands_pre_soft  = Counter()

        start_bands_intra_total = Counter()
        start_bands_intra_any   = Counter()
        start_bands_intra_hard  = Counter()
        start_bands_intra_soft  = Counter()
        norm_bands_intra_any   = Counter()
        norm_bands_intra_hard  = Counter()
        norm_bands_intra_soft  = Counter()

        start_bands_post_total = Counter()
        start_bands_post_any   = Counter()
        start_bands_post_hard  = Counter()
        start_bands_post_soft  = Counter()
        norm_bands_post_any   = Counter()
        norm_bands_post_hard  = Counter()
        norm_bands_post_soft  = Counter()

        timebands_pre_by_class_sign = make_timeband_map(["blue", "ark", "print", "open", "global"])
        timebands_intra_by_class_sign = make_timeband_map(["intra"])
        timebands_post_by_class_sign = make_timeband_map(["post"])

        last3_examples = make_last3_map(["blue", "ark", "print", "open", "global", "intra", "post"])

        recent_days = deque(maxlen=10)
        recent_by_day = {}
        last5_print_days_pos = deque(maxlen=5)
        last5_print_days_neg = deque(maxlen=5)
        last5_peak_days_pos = deque(maxlen=5)
        last5_peak_days_neg = deque(maxlen=5)

        hard_delay_sum = Counter()
        hard_delay_cnt = Counter()

        global_norm_peak_sum = {"pos": 0.0, "neg": 0.0}
        global_norm_peak_cnt = {"pos": 0, "neg": 0}

        open_series_by_day = {}

    # ---------------- common utils ----------------

    def parse_dt(x):
        """Уніфіковано привести вхід до datetime або None.
        Підтримує: datetime, pd.Timestamp, ISO-рядки; повертає Python datetime (може бути tz-aware) або None.
        """
        try:
            if x is None:
                return None
            if isinstance(x, datetime):
                return x
            # pandas поверне pd.Timestamp; встановлюємо utc=True щоб уникнути неоднозначностей
            ts = pd.to_datetime(x, errors="coerce", utc=True)
            if pd.isna(ts):
                return None
            return ts.to_pydatetime()
        except Exception:
            return None

    
    def push_last3_example(class_key, sign_key, kind, start_dt, end_dt, start_dev, end_dev, peak_dev,
                           start_stock, end_stock, start_bench, end_bench, start_time=None, end_time=None):
        d = start_dt.date().isoformat() if isinstance(start_dt, datetime) else None
        last3_examples[class_key][sign_key].appendleft({
            "date": d,
            "dt": _dt_iso(start_dt),
            "kind": kind,  # "hard"/"soft"
            "start_time": start_time,
            "end_time": end_time,
            "start_dev": _json_safe(start_dev),
            "peak_dev": _json_safe(peak_dev),
            "end_dev": _json_safe(end_dev),
            "stock_start": _json_safe(start_stock),
            "stock_end": _json_safe(end_stock),
            "bench_start": _json_safe(start_bench),
            "bench_end": _json_safe(end_bench),
        })

    def update_sigma_bins(map_ref, class_key, sign_key, abs_peak_sigma, outcome_kind):
        b = sigma_bin(abs_peak_sigma)
        if b is None:
            return
        st = map_ref[class_key][sign_key][b]
        st["total"] += 1
        if outcome_kind not in ("hard", "soft", "none"):
            outcome_kind = "none"
        st[outcome_kind] += 1

    def update_bench_bins(map_ref, class_key, which, sign_key, bench_value, outcome_kind):
        b = bench_bin(bench_value)
        if b is None:
            return
        st = map_ref[class_key][which][sign_key][b]
        st["total"] += 1
        if outcome_kind not in ("hard", "soft", "none"):
            outcome_kind = "none"
        st[outcome_kind] += 1

    def update_timeband_by_class_sign(map_ref, class_key, which, sign_key, band_key, outcome_kind):
        if not band_key:
            return
        st = map_ref[class_key][which][sign_key][band_key]
        st["total"] += 1
        if outcome_kind not in ("hard", "soft", "none"):
            outcome_kind = "none"
        st[outcome_kind] += 1
        _ = st["hard"]; _ = st["soft"]; _ = st["none"]

    def class_rates(counter: Counter):
        total = int(sum(counter.values()))
        hard = int(counter.get("hard", 0))
        soft = int(counter.get("soft", 0))
        none = int(counter.get("none", 0))
        any_ = hard + soft
        return {
            "total": total,
            "hard": hard,
            "soft": soft,
            "none": none,
            "rate_any": (any_ / total) if total else None,
            "rate_hard": (hard / total) if total else None,
            "rate_soft": (soft / total) if total else None,
            "hard_share_in_norm": (hard / (hard + soft)) if (hard + soft) else None,
        }

    def add_hard_delay(key: str, start_dt: datetime, hard_dt: datetime):
        if isinstance(start_dt, datetime) and isinstance(hard_dt, datetime) and hard_dt >= start_dt:
            hard_delay_sum[key] += (hard_dt - start_dt).total_seconds()
            hard_delay_cnt[key] += 1

    def avg_hard_delay(key: str):
        c = int(hard_delay_cnt.get(key, 0))
        if c <= 0:
            return None
        return float(hard_delay_sum.get(key, 0.0)) / c

    # ---------------- PRE event state ----------------
    pre_active = False
    pre_id = 0

    pre_start_dt = None
    pre_start_dev = np.nan
    pre_start_sign = 0
    pre_start_stock = np.nan
    pre_start_bench = np.nan

    # PRE peak frozen until 09:29 (for ARK/PRINT/OPEN/GLOBAL)
    pre_peak_abs = 0.0
    pre_peak_signed = 0.0
    pre_peak_dt = None
    pre_peak_stock = np.nan
    pre_peak_bench = np.nan

    pre_post_peak_low_abs = np.inf

    # BLUE peak frozen until 03:59 (for BLUE soft)
    blue_peak_abs = 0.0
    blue_peak_signed = 0.0
    blue_peak_dt = None
    blue_peak_stock = np.nan
    blue_peak_bench = np.nan

    blue_hard_dt = None
    blue_hard_val = np.nan
    blue_hard_stock = np.nan
    blue_hard_bench = np.nan

    blue_soft_found = False
    blue_soft_dt = None
    blue_soft_val = np.nan
    blue_soft_stock = np.nan
    blue_soft_bench = np.nan

    ark_hard_dt = None
    ark_hard_val = np.nan
    ark_hard_stock = np.nan
    ark_hard_bench = np.nan

    ark_soft_found = False
    ark_soft_dt = None
    ark_soft_val = np.nan
    ark_soft_stock = np.nan
    ark_soft_bench = np.nan

    print_first_dt = None
    print_first_val = np.nan
    print_first_stock = np.nan
    print_first_bench = np.nan

    open_hard_dt = None
    open_hard_val = np.nan
    open_hard_stock = np.nan
    open_hard_bench = np.nan

    open_soft_found = False
    open_soft_dt = None
    open_soft_val = np.nan
    open_soft_stock = np.nan
    open_soft_bench = np.nan

    def reset_pre_event():
        nonlocal pre_active, pre_start_dt, pre_start_dev, pre_start_sign, pre_start_stock, pre_start_bench
        nonlocal pre_peak_abs, pre_peak_signed, pre_peak_dt, pre_peak_stock, pre_peak_bench
        nonlocal pre_post_peak_low_abs
        nonlocal blue_peak_abs, blue_peak_signed, blue_peak_dt, blue_peak_stock, blue_peak_bench
        nonlocal blue_hard_dt, blue_hard_val, blue_hard_stock, blue_hard_bench
        nonlocal blue_soft_found, blue_soft_dt, blue_soft_val, blue_soft_stock, blue_soft_bench
        nonlocal ark_hard_dt, ark_hard_val, ark_hard_stock, ark_hard_bench
        nonlocal ark_soft_found, ark_soft_dt, ark_soft_val, ark_soft_stock, ark_soft_bench
        nonlocal print_first_dt, print_first_val, print_first_stock, print_first_bench
        nonlocal open_hard_dt, open_hard_val, open_hard_stock, open_hard_bench
        nonlocal open_soft_found, open_soft_dt, open_soft_val, open_soft_stock, open_soft_bench

        pre_active = False
        pre_start_dt = None
        pre_start_dev = np.nan
        pre_start_sign = 0
        pre_start_stock = np.nan
        pre_start_bench = np.nan

        pre_peak_abs = 0.0
        pre_peak_signed = 0.0
        pre_peak_dt = None
        pre_peak_stock = np.nan
        pre_peak_bench = np.nan

        pre_post_peak_low_abs = np.inf

        blue_peak_abs = 0.0
        blue_peak_signed = 0.0
        blue_peak_dt = None
        blue_peak_stock = np.nan
        blue_peak_bench = np.nan

        blue_hard_dt = None
        blue_hard_val = np.nan
        blue_hard_stock = np.nan
        blue_hard_bench = np.nan

        blue_soft_found = False
        blue_soft_dt = None
        blue_soft_val = np.nan
        blue_soft_stock = np.nan
        blue_soft_bench = np.nan

        ark_hard_dt = None
        ark_hard_val = np.nan
        ark_hard_stock = np.nan
        ark_hard_bench = np.nan

        ark_soft_found = False
        ark_soft_dt = None
        ark_soft_val = np.nan
        ark_soft_stock = np.nan
        ark_soft_bench = np.nan

        print_first_dt = None
        print_first_val = np.nan
        print_first_stock = np.nan
        print_first_bench = np.nan

        open_hard_dt = None
        open_hard_val = np.nan
        open_hard_stock = np.nan
        open_hard_bench = np.nan

        open_soft_found = False
        open_soft_dt = None
        open_soft_val = np.nan
        open_soft_stock = np.nan
        open_soft_bench = np.nan

    def start_pre_event(dt_now, dev_now, stock_pct, bench_pct):
        nonlocal pre_active, pre_start_dt, pre_start_dev, pre_start_sign, pre_start_stock, pre_start_bench
        nonlocal pre_peak_abs, pre_peak_signed, pre_peak_dt, pre_peak_stock, pre_peak_bench
        nonlocal blue_peak_abs, blue_peak_signed, blue_peak_dt, blue_peak_stock, blue_peak_bench

        pre_active = True
        pre_start_dt = dt_now
        pre_start_dev = float(dev_now)
        pre_start_sign = 1 if float(dev_now) >= 0 else -1
        pre_start_stock = stock_pct
        pre_start_bench = bench_pct

        # PRE peak init (shared)
        pre_peak_abs = abs(float(dev_now))
        pre_peak_signed = float(dev_now)
        pre_peak_dt = dt_now
        pre_peak_stock = stock_pct
        pre_peak_bench = bench_pct

        # BLUE peak init (so BLUE soft works even if event starts in BLUE)
        blue_peak_abs = abs(float(dev_now))
        blue_peak_signed = float(dev_now)
        blue_peak_dt = dt_now
        blue_peak_stock = stock_pct
        blue_peak_bench = bench_pct

    def pre_sign_key():
        return "pos" if pre_start_sign > 0 else "neg"

    def classify_print_with_frozen_peak(first_val):
        if not is_finite_num(first_val):
            return "none"
        a = abs(float(first_val))
        if a <= norm_thr:
            return "hard"
        if pre_peak_abs > 0 and a <= (float(pre_peak_abs) / float(soft_ratio)):
            return "soft"
        return "none"

    def classify_blue():
        if blue_hard_dt is not None and is_finite_num(blue_hard_val):
            return "hard"
        if blue_soft_found and blue_soft_dt is not None and is_finite_num(blue_soft_val):
            return "soft"
        return "none"

    def classify_ark():
        if ark_hard_dt is not None and is_finite_num(ark_hard_val):
            return "hard"
        if ark_soft_found and ark_soft_dt is not None and is_finite_num(ark_soft_val):
            return "soft"
        return "none"

    def classify_open():
        if open_hard_dt is not None and is_finite_num(open_hard_val):
            return "hard"
        if open_soft_found and open_soft_dt is not None and is_finite_num(open_soft_val):
            return "soft"
        return "none"

    def capture_open_series(dt_now: datetime, dev_now: float):
        if not isinstance(dt_now, datetime):
            return
        t = (dt_now.hour, dt_now.minute)
        if not in_range(t, OPEN_FROM, OPEN_TO):
            return

        day_str = dt_now.date().isoformat()
        store = open_series_by_day.get(day_str)
        if store is None:
            store = {}
            open_series_by_day[day_str] = store

        sec = max(1, int(open_series_downsample_seconds))
        bucket_epoch = int(dt_now.timestamp() // sec) * sec
        # use pandas to create tz-aware Timestamp consistently
        bucket_dt = pd.Timestamp(bucket_epoch, unit="s", tz=dt_now.tzinfo).to_pydatetime()
        store[bucket_dt.isoformat()] = float(dev_now)

    def pre_process_tick(dt_now, dev_now, stock_pct, bench_pct):
        nonlocal pre_peak_abs, pre_peak_signed, pre_peak_dt, pre_peak_stock, pre_peak_bench
        nonlocal pre_post_peak_low_abs

        nonlocal blue_peak_abs, blue_peak_signed, blue_peak_dt, blue_peak_stock, blue_peak_bench
        nonlocal blue_hard_dt, blue_hard_val, blue_hard_stock, blue_hard_bench
        nonlocal blue_soft_found, blue_soft_dt, blue_soft_val, blue_soft_stock, blue_soft_bench

        nonlocal ark_hard_dt, ark_hard_val, ark_hard_stock, ark_hard_bench
        nonlocal ark_soft_found, ark_soft_dt, ark_soft_val, ark_soft_stock, ark_soft_bench

        nonlocal print_first_dt, print_first_val, print_first_stock, print_first_bench

        nonlocal open_hard_dt, open_hard_val, open_hard_stock, open_hard_bench
        nonlocal open_soft_found, open_soft_dt, open_soft_val, open_soft_stock, open_soft_bench

        t = (dt_now.hour, dt_now.minute)
        cur_abs = abs(float(dev_now))

        # capture open series (kept)
        capture_open_series(dt_now, float(dev_now))

        # ---------- BLUE processing inside 00:01–03:59 (parallel) ----------
        if in_range(t, BLUE_FROM, BLUE_TO):
            # BLUE peak frozen until 03:59
            if cur_abs > float(blue_peak_abs):
                blue_peak_abs = float(cur_abs)
                blue_peak_signed = float(dev_now)
                blue_peak_dt = dt_now
                blue_peak_stock = stock_pct
                blue_peak_bench = bench_pct
                if blue_hard_dt is None:
                    blue_soft_found = False
                    blue_soft_dt = None
                    blue_soft_val = np.nan
                    blue_soft_stock = np.nan
                    blue_soft_bench = np.nan

            # BLUE HARD
            if blue_hard_dt is None and cur_abs <= norm_thr:
                blue_hard_dt = dt_now
                blue_hard_val = float(dev_now)
                blue_hard_stock = stock_pct
                blue_hard_bench = bench_pct

            # BLUE SOFT only if hard not yet, after BLUE peak
            if (blue_hard_dt is None) and isinstance(blue_peak_dt, datetime) and (dt_now >= blue_peak_dt) and blue_peak_abs > 0:
                if cur_abs <= (float(blue_peak_abs) / float(soft_ratio)):
                    if (not blue_soft_found) or (dt_now < blue_soft_dt):
                        blue_soft_found = True
                        blue_soft_dt = dt_now
                        blue_soft_val = float(dev_now)
                        blue_soft_stock = stock_pct
                        blue_soft_bench = bench_pct

        # ---------- PRE peak (for ARK/PRINT/OPEN/GLOBAL) frozen until 09:29 ----------
        if t <= ARK_TO:
            if cur_abs > float(pre_peak_abs):
                pre_peak_abs = float(cur_abs)
                pre_peak_signed = float(dev_now)
                pre_peak_dt = dt_now
                pre_peak_stock = stock_pct
                pre_peak_bench = bench_pct
                if ark_hard_dt is None:
                    ark_soft_found = False
                    ark_soft_dt = None
                    ark_soft_val = np.nan
                    ark_soft_stock = np.nan
                    ark_soft_bench = np.nan

        # post-peak low abs after peak (kept)
        if isinstance(pre_peak_dt, datetime) and dt_now >= pre_peak_dt:
            if cur_abs < pre_post_peak_low_abs:
                pre_post_peak_low_abs = cur_abs

        # ARK HARD in [start..09:29]
        if (ark_hard_dt is None) and (t <= ARK_TO):
            if cur_abs <= norm_thr:
                ark_hard_dt = dt_now
                ark_hard_val = float(dev_now)
                ark_hard_stock = stock_pct
                ark_hard_bench = bench_pct

        # ARK SOFT only if hard failed, in [peak_dt..09:29]
        if (ark_hard_dt is None) and (t <= ARK_TO) and isinstance(pre_peak_dt, datetime):
            if dt_now >= pre_peak_dt and pre_peak_abs > 0:
                if cur_abs <= (float(pre_peak_abs) / float(soft_ratio)):
                    if (not ark_soft_found) or (dt_now < ark_soft_dt):
                        ark_soft_found = True
                        ark_soft_dt = dt_now
                        ark_soft_val = float(dev_now)
                        ark_soft_stock = stock_pct
                        ark_soft_bench = bench_pct

        # PRINT first tick only in [09:30..09:35]
        if (print_first_dt is None) and in_range(t, PRINT_FROM, PRINT_TO):
            print_first_dt = dt_now
            print_first_val = float(dev_now)
            print_first_stock = stock_pct
            print_first_bench = bench_pct

        # OPEN scan in [09:31..09:40]
        if in_range(t, OPEN_FROM, OPEN_TO):
            if open_hard_dt is None and cur_abs <= norm_thr:
                open_hard_dt = dt_now
                open_hard_val = float(dev_now)
                open_hard_stock = stock_pct
                open_hard_bench = bench_pct

            if open_hard_dt is None and pre_peak_abs > 0:
                gate_dt = pre_peak_dt if isinstance(pre_peak_dt, datetime) else dt_now
                open_gate = dt_now.replace(hour=OPEN_FROM[0], minute=OPEN_FROM[1], second=0, microsecond=0)
                if gate_dt < open_gate:
                    gate_dt = open_gate
                if dt_now >= gate_dt:
                    if cur_abs <= (float(pre_peak_abs) / float(soft_ratio)):
                        if (not open_soft_found) or (dt_now < open_soft_dt):
                            open_soft_found = True
                            open_soft_dt = dt_now
                            open_soft_val = float(dev_now)
                            open_soft_stock = stock_pct
                            open_soft_bench = bench_pct

    def _ensure_recent_day(day_str: str):
        if day_str not in recent_by_day:
            recent_by_day[day_str] = {"print": None, "peak": None}
        if (len(recent_days) == 0) or (recent_days[0] != day_str):
            if day_str in recent_days:
                recent_days.remove(day_str)
            recent_days.appendleft(day_str)

    def _update_recent_daily_print(day_str: str, print_dt: datetime, print_dev: float, peak_abs: float, peak_signed: float, first_sign: int):
        snap = recent_by_day.get(day_str)
        if snap is None:
            recent_by_day[day_str] = {"print": None, "peak": None}
            snap = recent_by_day[day_str]
        if snap["print"] is None:
            snap["print"] = {
                "dt": _dt_iso(print_dt),
                "dev": _json_safe(print_dev),
                "pre_peak_abs": _json_safe(peak_abs),
                "pre_peak_signed": _json_safe(peak_signed),
                "first_sign": int(first_sign),
            }

    def _update_recent_daily_peak(day_str: str, peak_dt: datetime, peak_abs: float, peak_signed: float, first_sign: int):
        snap = recent_by_day.get(day_str)
        if snap is None:
            recent_by_day[day_str] = {"print": None, "peak": None}
            snap = recent_by_day[day_str]
        cur = snap["peak"]
        if cur is None or (is_finite_num(peak_abs) and float(peak_abs) > float(cur.get("sigma_abs", 0.0) or 0.0)):
            snap["peak"] = {
                "dt": _dt_iso(peak_dt),
                "sigma_abs": _json_safe(peak_abs),
                "sigma_signed": _json_safe(peak_signed),
                "first_sign": int(first_sign),
            }

    def finalize_pre_event(reason="window_end"):
        nonlocal pre_active, pre_id
        nonlocal global_norm_peak_sum, global_norm_peak_cnt

        if not pre_active:
            return

        blue_status  = classify_blue()
        ark_status   = classify_ark()
        print_status = classify_print_with_frozen_peak(print_first_val)
        open_status  = classify_open()

        global_label = compute_global_label(blue_status, ark_status, print_status, open_status)
        if global_label.endswith("_HARD"):
            global_kind = "hard"
        elif global_label.endswith("_SOFT"):
            global_kind = "soft"
        else:
            global_kind = "none"

        sk = pre_sign_key()

        # OLD start band counters (kept) — any/hard/soft across all pre classes (OR semantics)
        sb_total = floor_to_band(pre_start_dt, start_band_minutes)
        if sb_total:
            start_bands_pre_total[sb_total] += 1
            if (blue_status != "none") or (ark_status != "none") or (print_status != "none") or (open_status != "none"):
                start_bands_pre_any[sb_total] += 1
            if (blue_status == "hard") or (ark_status == "hard") or (print_status == "hard") or (open_status == "hard"):
                start_bands_pre_hard[sb_total] += 1
            if (blue_status == "soft") or (ark_status == "soft") or (print_status == "soft") or (open_status == "soft"):
                start_bands_pre_soft[sb_total] += 1

        # NEW per-class-sign start band counters
        if sb_total:
            update_timeband_by_class_sign(timebands_pre_by_class_sign, "blue", "start", sk, sb_total, blue_status)
            update_timeband_by_class_sign(timebands_pre_by_class_sign, "ark", "start", sk, sb_total, ark_status)
            update_timeband_by_class_sign(timebands_pre_by_class_sign, "print", "start", sk, sb_total, print_status)
            update_timeband_by_class_sign(timebands_pre_by_class_sign, "open", "start", sk, sb_total, open_status)
            update_timeband_by_class_sign(timebands_pre_by_class_sign, "global", "start", sk, sb_total, global_kind)

        # counts (classes independent)
        counts_pre["blue"][blue_status] += 1
        counts_pre["ark"][ark_status] += 1
        counts_pre["print"][print_status] += 1
        counts_pre["open"][open_status] += 1
        counts_pre["global"][global_kind] += 1
        global_labels_counter[global_label] += 1

        # HARD delays (kept + blue)
        if blue_status == "hard":
            add_hard_delay("blue", pre_start_dt, blue_hard_dt)
        if ark_status == "hard":
            add_hard_delay("ark", pre_start_dt, ark_hard_dt)
        if print_status == "hard":
            add_hard_delay("print", pre_start_dt, print_first_dt)
        if open_status == "hard":
            add_hard_delay("open", pre_start_dt, open_hard_dt)
        if global_kind == "hard":
            # delay is delay to the winning hard in GLOBAL
            if global_label.startswith("BLUE_"):
                add_hard_delay("global", pre_start_dt, blue_hard_dt)
            elif global_label.startswith("ARK_"):
                add_hard_delay("global", pre_start_dt, ark_hard_dt)
            elif global_label.startswith("PRINT_"):
                add_hard_delay("global", pre_start_dt, print_first_dt)
            elif global_label.startswith("OPEN_"):
                add_hard_delay("global", pre_start_dt, open_hard_dt)

        # sigma bins:
        # - BLUE uses BLUE peak (frozen to 03:59)
        # - others use PRE peak (frozen to 09:29)
        update_sigma_bins(sigma_bins_pre, "blue", sk, blue_peak_abs, blue_status)
        update_sigma_bins(sigma_bins_pre, "ark", sk, pre_peak_abs, ark_status)
        update_sigma_bins(sigma_bins_pre, "print", sk, pre_peak_abs, print_status)
        update_sigma_bins(sigma_bins_pre, "open", sk, pre_peak_abs, open_status)
        update_sigma_bins(sigma_bins_pre, "global", sk, pre_peak_abs, global_kind)  # global is modeled on PRE peak scale

        # bench bins start+peak
        update_bench_bins(bench_bins_pre, "blue", "start", sk, pre_start_bench, blue_status)
        update_bench_bins(bench_bins_pre, "blue", "peak",  sk, blue_peak_bench, blue_status)

        for cls, status in (("ark", ark_status), ("print", print_status), ("open", open_status), ("global", global_kind)):
            update_bench_bins(bench_bins_pre, cls, "start", sk, pre_start_bench, status)
            update_bench_bins(bench_bins_pre, cls, "peak",  sk, pre_peak_bench, status)

        # norm dt per class (for norm bench bins + norm timebands)
        def cls_norm_dt(cls):
            if cls == "blue":
                return blue_hard_dt if blue_status == "hard" else (blue_soft_dt if blue_status == "soft" else None)
            if cls == "ark":
                return ark_hard_dt if ark_status == "hard" else (ark_soft_dt if ark_status == "soft" else None)
            if cls == "print":
                return print_first_dt if print_status != "none" else None
            if cls == "open":
                return open_hard_dt if open_status == "hard" else (open_soft_dt if open_status == "soft" else None)
            if cls == "global":
                if global_label.startswith("BLUE_"):
                    return cls_norm_dt("blue")
                if global_label.startswith("ARK_"):
                    return cls_norm_dt("ark")
                if global_label.startswith("PRINT_"):
                    return cls_norm_dt("print")
                if global_label.startswith("OPEN_"):
                    return cls_norm_dt("open")
                return None
            return None

        def cls_outcome(cls):
            if cls == "blue": return blue_status
            if cls == "ark": return ark_status
            if cls == "print": return print_status
            if cls == "open": return open_status
            if cls == "global": return global_kind
            return "none"

        def cls_peak_signed(cls):
            if cls == "blue": return blue_peak_signed
            return pre_peak_signed

        def cls_end_fields(cls, status):
            if cls == "blue":
                if status == "hard":
                    return blue_hard_val, blue_hard_stock, blue_hard_bench
                if status == "soft":
                    return blue_soft_val, blue_soft_stock, blue_soft_bench
                return np.nan, np.nan, np.nan
            if cls == "ark":
                if status == "hard":
                    return ark_hard_val, ark_hard_stock, ark_hard_bench
                if status == "soft":
                    return ark_soft_val, ark_soft_stock, ark_soft_bench
                return np.nan, np.nan, np.nan
            if cls == "print":
                if status != "none":
                    return print_first_val, print_first_stock, print_first_bench
                return np.nan, np.nan, np.nan
            if cls == "open":
                if status == "hard":
                    return open_hard_val, open_hard_stock, open_hard_bench
                if status == "soft":
                    return open_soft_val, open_soft_stock, open_soft_bench
                return np.nan, np.nan, np.nan
            if cls == "global":
                if global_label.startswith("BLUE_"):
                    return cls_end_fields("blue", blue_status)
                if global_label.startswith("ARK_"):
                    return cls_end_fields("ark", ark_status)
                if global_label.startswith("PRINT_"):
                    return cls_end_fields("print", print_status)
                if global_label.startswith("OPEN_"):
                    return cls_end_fields("open", open_status)
                return np.nan, np.nan, np.nan
            return np.nan, np.nan, np.nan

        # norm bins + bench norm + ✅ last3 pushes (hard/soft only)
        for cls in ("blue", "ark", "print", "open", "global"):
            ndt = cls_norm_dt(cls)
            status = cls_outcome(cls)
            if isinstance(ndt, datetime):
                b = floor_to_band(ndt, norm_band_minutes)
                if b:
                    # OLD norm counters kept (any/hard/soft only)
                    if status != "none":
                        norm_bands_pre_any[b] += 1
                    if status == "hard":
                        norm_bands_pre_hard[b] += 1
                    if status == "soft":
                        norm_bands_pre_soft[b] += 1

                    # NEW per-class-sign norm bins
                    update_timeband_by_class_sign(timebands_pre_by_class_sign, cls, "norm", sk, b, status)

                    # bench norm bin value:
                    if cls == "blue":
                        end_bench_val = blue_hard_bench if status == "hard" else (blue_soft_bench if status == "soft" else np.nan)
                        update_bench_bins(bench_bins_pre, cls, "norm", sk, end_bench_val, status)
                    else:
                        update_bench_bins(bench_bins_pre, cls, "norm", sk, pre_peak_bench, status)

                # last3 only for normalized statuses
                if status in ("hard", "soft"):
                    end_dev, end_stock, end_bench = cls_end_fields(cls, status)
                    push_last3_example(
                        cls, sk, status,
                        pre_start_dt, ndt,
                        pre_start_dev, end_dev, cls_peak_signed(cls),
                        pre_start_stock, end_stock,
                        pre_start_bench, end_bench,
                        start_time=pre_start_dt.strftime("%H:%M") if isinstance(pre_start_dt, datetime) else None,
                        end_time=ndt.strftime("%H:%M") if isinstance(ndt, datetime) else None,
                    )

        # global mean peak for normalized — use BLUE peak if global label is BLUE
        if global_kind != "none":
            if global_label.startswith("BLUE_") and is_finite_num(blue_peak_abs):
                global_norm_peak_sum[sk] += float(blue_peak_abs)
                global_norm_peak_cnt[sk] += 1
            elif is_finite_num(pre_peak_abs):
                global_norm_peak_sum[sk] += float(pre_peak_abs)
                global_norm_peak_cnt[sk] += 1

        # DAILY RECENT (kept): based on PRE peak (09:29-frozen)
        if isinstance(pre_start_dt, datetime):
            day_str = pre_start_dt.date().isoformat()
            _ensure_recent_day(day_str)
            _update_recent_daily_peak(day_str, pre_peak_dt, pre_peak_abs, pre_peak_signed, int(pre_start_sign))
            if isinstance(print_first_dt, datetime) and is_finite_num(print_first_val):
                _update_recent_daily_print(
                    day_str,
                    print_first_dt,
                    float(print_first_val),
                    float(pre_peak_abs),
                    float(pre_peak_signed),
                    int(pre_start_sign),
                )

        # rebuild last5 deques from recent_days (kept)
        last5_print_days_pos.clear()
        last5_print_days_neg.clear()
        last5_peak_days_pos.clear()
        last5_peak_days_neg.clear()
        for d in list(recent_days):
            snap = recent_by_day.get(d)
            if not snap:
                continue
            pk = snap.get("peak")
            pr = snap.get("print")
            if pk and is_finite_num(pk.get("sigma_abs")):
                if int(pk.get("first_sign", 1)) > 0:
                    last5_peak_days_pos.append(float(pk["sigma_abs"]))
                else:
                    last5_peak_days_neg.append(float(pk["sigma_abs"]))
            if pr and is_finite_num(pr.get("dev")):
                if int(pr.get("first_sign", 1)) > 0:
                    last5_print_days_pos.append(float(pr["dev"]))
                else:
                    last5_print_days_neg.append(float(pr["dev"]))
            if (
                len(last5_print_days_pos) >= 5
                and len(last5_print_days_neg) >= 5
                and len(last5_peak_days_pos) >= 5
                and len(last5_peak_days_neg) >= 5
            ):
                break

        if include_events_pre:
            pre_events_buf.append({
                "pre_id": int(pre_id),
                "reason_finalized": reason,
                "start": {"dt": _dt_iso(pre_start_dt), "dev": _json_safe(pre_start_dev), "sign": int(pre_start_sign),
                          "stock_pct": _json_safe(pre_start_stock), "bench_pct": _json_safe(pre_start_bench)},
                "pre_peak_frozen": {"dt": _dt_iso(pre_peak_dt), "abs": _json_safe(pre_peak_abs), "signed": _json_safe(pre_peak_signed),
                                    "bin": sigma_bin(pre_peak_abs),
                                    "stock_pct": _json_safe(pre_peak_stock), "bench_pct": _json_safe(pre_peak_bench)},
                "blue_peak_frozen": {"dt": _dt_iso(blue_peak_dt), "abs": _json_safe(blue_peak_abs), "signed": _json_safe(blue_peak_signed),
                                     "bin": sigma_bin(blue_peak_abs),
                                     "stock_pct": _json_safe(blue_peak_stock), "bench_pct": _json_safe(blue_peak_bench)},
                "blue": {"status": blue_status},
                "ark": {"status": ark_status},
                "print": {"status": print_status},
                "open": {"status": open_status},
                "global": {"label": global_label, "kind": global_kind},
            })

        pre_id += 1
        reset_pre_event()

    # ---------------- INTRA event state ----------------
    intra_active = False
    intra_id = 0

    intra_start_dt = None
    intra_start_dev = np.nan
    intra_start_sign = 0
    intra_start_stock = np.nan
    intra_start_bench = np.nan

    intra_peak_abs = 0.0
    intra_peak_signed = 0.0
    intra_peak_dt = None
    intra_peak_stock = np.nan
    intra_peak_bench = np.nan

    intra_hard_dt = None
    intra_hard_val = np.nan
    intra_hard_stock = np.nan
    intra_hard_bench = np.nan

    intra_soft_found = False
    intra_soft_dt = None
    intra_soft_val = np.nan
    intra_soft_stock = np.nan
    intra_soft_bench = np.nan

    def reset_intra_event():
        nonlocal intra_active, intra_start_dt, intra_start_dev, intra_start_sign, intra_start_stock, intra_start_bench
        nonlocal intra_peak_abs, intra_peak_signed, intra_peak_dt, intra_peak_stock, intra_peak_bench
        nonlocal intra_hard_dt, intra_hard_val, intra_hard_stock, intra_hard_bench
        nonlocal intra_soft_found, intra_soft_dt, intra_soft_val, intra_soft_stock, intra_soft_bench

        intra_active = False
        intra_start_dt = None
        intra_start_dev = np.nan
        intra_start_sign = 0
        intra_start_stock = np.nan
        intra_start_bench = np.nan

        intra_peak_abs = 0.0
        intra_peak_signed = 0.0
        intra_peak_dt = None
        intra_peak_stock = np.nan
        intra_peak_bench = np.nan

        intra_hard_dt = None
        intra_hard_val = np.nan
        intra_hard_stock = np.nan
        intra_hard_bench = np.nan

        intra_soft_found = False
        intra_soft_dt = None
        intra_soft_val = np.nan
        intra_soft_stock = np.nan
        intra_soft_bench = np.nan

    def start_intra_event(dt_now, dev_now, stock_pct, bench_pct):
        nonlocal intra_active, intra_start_dt, intra_start_dev, intra_start_sign, intra_start_stock, intra_start_bench
        nonlocal intra_peak_abs, intra_peak_signed, intra_peak_dt, intra_peak_stock, intra_peak_bench

        intra_active = True
        intra_start_dt = dt_now
        intra_start_dev = float(dev_now)
        intra_start_sign = 1 if float(dev_now) >= 0 else -1
        intra_start_stock = stock_pct
        intra_start_bench = bench_pct

        intra_peak_abs = abs(float(dev_now))
        intra_peak_signed = float(dev_now)
        intra_peak_dt = dt_now
        intra_peak_stock = stock_pct
        intra_peak_bench = bench_pct

    def intra_sign_key():
        return "pos" if intra_start_sign > 0 else "neg"

    def intra_process_tick(dt_now, dev_now, stock_pct, bench_pct):
        nonlocal intra_peak_abs, intra_peak_signed, intra_peak_dt, intra_peak_stock, intra_peak_bench
        nonlocal intra_hard_dt, intra_hard_val, intra_hard_stock, intra_hard_bench
        nonlocal intra_soft_found, intra_soft_dt, intra_soft_val, intra_soft_stock, intra_soft_bench

        cur_abs = abs(float(dev_now))

        if cur_abs > float(intra_peak_abs):
            intra_peak_abs = float(cur_abs)
            intra_peak_signed = float(dev_now)
            intra_peak_dt = dt_now
            intra_peak_stock = stock_pct
            intra_peak_bench = bench_pct
            if intra_hard_dt is None:
                intra_soft_found = False
                intra_soft_dt = None
                intra_soft_val = np.nan
                intra_soft_stock = np.nan
                intra_soft_bench = np.nan

        t = (dt_now.hour, dt_now.minute)
        if not in_range(t, INTRA_FROM, INTRA_TO):
            return

        if intra_hard_dt is None and cur_abs <= norm_thr:
            intra_hard_dt = dt_now
            intra_hard_val = float(dev_now)
            intra_hard_stock = stock_pct
            intra_hard_bench = bench_pct

        if intra_hard_dt is None and isinstance(intra_peak_dt, datetime) and intra_peak_abs > 0:
            if dt_now >= intra_peak_dt and cur_abs <= (float(intra_peak_abs) / float(soft_ratio)):
                if (not intra_soft_found) or (dt_now < intra_soft_dt):
                    intra_soft_found = True
                    intra_soft_dt = dt_now
                    intra_soft_val = float(dev_now)
                    intra_soft_stock = stock_pct
                    intra_soft_bench = bench_pct

    def finalize_intra_event(reason="window_end"):
        nonlocal intra_active, intra_id
        if not intra_active:
            return

        if intra_hard_dt is not None:
            status = "hard"
            end_dt = intra_hard_dt
            add_hard_delay("intra", intra_start_dt, intra_hard_dt)
        elif intra_soft_found and intra_soft_dt is not None:
            status = "soft"
            end_dt = intra_soft_dt
        else:
            status = "none"
            end_dt = None

        sk = intra_sign_key()

        sb = floor_to_band(intra_start_dt, start_band_minutes)
        if sb:
            start_bands_intra_total[sb] += 1
            if status != "none":
                start_bands_intra_any[sb] += 1
            if status == "hard":
                start_bands_intra_hard[sb] += 1
            if status == "soft":
                start_bands_intra_soft[sb] += 1

            update_timeband_by_class_sign(timebands_intra_by_class_sign, "intra", "start", sk, sb, status)

        counts_intra["intra"][status] += 1

        update_sigma_bins(sigma_bins_intra, "intra", sk, intra_peak_abs, status)
        update_bench_bins(bench_bins_intra, "intra", "start", sk, intra_start_bench, status)
        update_bench_bins(bench_bins_intra, "intra", "peak",  sk, intra_peak_bench, status)

        if status != "none" and isinstance(end_dt, datetime):
            b = floor_to_band(end_dt, norm_band_minutes)
            if b:
                norm_bands_intra_any[b] += 1
                if status == "hard":
                    norm_bands_intra_hard[b] += 1
                if status == "soft":
                    norm_bands_intra_soft[b] += 1

                update_timeband_by_class_sign(timebands_intra_by_class_sign, "intra", "norm", sk, b, status)
                update_bench_bins(bench_bins_intra, "intra", "norm", sk, (intra_hard_bench if status == "hard" else intra_soft_bench), status)

            push_last3_example(
                "intra", sk, status,
                intra_start_dt, end_dt,
                intra_start_dev, (intra_hard_val if status == "hard" else intra_soft_val),
                intra_peak_signed,
                intra_start_stock, (intra_hard_stock if status == "hard" else intra_soft_stock),
                intra_start_bench, (intra_hard_bench if status == "hard" else intra_soft_bench),
                start_time=intra_start_dt.strftime("%H:%M") if isinstance(intra_start_dt, datetime) else None,
                end_time=end_dt.strftime("%H:%M") if isinstance(end_dt, datetime) else None,
            )

        if include_events_intra:
            intra_events_buf.append({
                "intra_id": int(intra_id),
                "reason_finalized": reason,
                "start": {"dt": _dt_iso(intra_start_dt), "dev": _json_safe(intra_start_dev), "sign": int(intra_start_sign)},
                "peak":  {"dt": _dt_iso(intra_peak_dt), "abs": _json_safe(intra_peak_abs), "signed": _json_safe(intra_peak_signed)},
                "status": status,
            })

        intra_id += 1
        reset_intra_event()

    # ---------------- POST event state ----------------
    post_active = False
    post_id = 0

    post_start_dt = None
    post_start_dev = np.nan
    post_start_sign = 0
    post_start_stock = np.nan
    post_start_bench = np.nan

    post_peak_abs = 0.0
    post_peak_signed = 0.0
    post_peak_dt = None
    post_peak_stock = np.nan
    post_peak_bench = np.nan

    post_hard_dt = None
    post_hard_val = np.nan
    post_hard_stock = np.nan
    post_hard_bench = np.nan

    post_soft_found = False
    post_soft_dt = None
    post_soft_val = np.nan
    post_soft_stock = np.nan
    post_soft_bench = np.nan

    def reset_post_event():
        nonlocal post_active, post_start_dt, post_start_dev, post_start_sign, post_start_stock, post_start_bench
        nonlocal post_peak_abs, post_peak_signed, post_peak_dt, post_peak_stock, post_peak_bench
        nonlocal post_hard_dt, post_hard_val, post_hard_stock, post_hard_bench
        nonlocal post_soft_found, post_soft_dt, post_soft_val, post_soft_stock, post_soft_bench

        post_active = False
        post_start_dt = None
        post_start_dev = np.nan
        post_start_sign = 0
        post_start_stock = np.nan
        post_start_bench = np.nan

        post_peak_abs = 0.0
        post_peak_signed = 0.0
        post_peak_dt = None
        post_peak_stock = np.nan
        post_peak_bench = np.nan

        post_hard_dt = None
        post_hard_val = np.nan
        post_hard_stock = np.nan
        post_hard_bench = np.nan

        post_soft_found = False
        post_soft_dt = None
        post_soft_val = np.nan
        post_soft_stock = np.nan
        post_soft_bench = np.nan

    def start_post_event(dt_now, dev_now, stock_pct, bench_pct):
        nonlocal post_active, post_start_dt, post_start_dev, post_start_sign, post_start_stock, post_start_bench
        nonlocal post_peak_abs, post_peak_signed, post_peak_dt, post_peak_stock, post_peak_bench

        post_active = True
        post_start_dt = dt_now
        post_start_dev = float(dev_now)
        post_start_sign = 1 if float(dev_now) >= 0 else -1
        post_start_stock = stock_pct
        post_start_bench = bench_pct

        post_peak_abs = abs(float(dev_now))
        post_peak_signed = float(dev_now)
        post_peak_dt = dt_now
        post_peak_stock = stock_pct
        post_peak_bench = bench_pct

    def post_sign_key():
        return "pos" if post_start_sign > 0 else "neg"

    def post_process_tick(dt_now, dev_now, stock_pct, bench_pct):
        nonlocal post_peak_abs, post_peak_signed, post_peak_dt, post_peak_stock, post_peak_bench
        nonlocal post_hard_dt, post_hard_val, post_hard_stock, post_hard_bench
        nonlocal post_soft_found, post_soft_dt, post_soft_val, post_soft_stock, post_soft_bench

        cur_abs = abs(float(dev_now))

        if cur_abs > float(post_peak_abs):
            post_peak_abs = float(cur_abs)
            post_peak_signed = float(dev_now)
            post_peak_dt = dt_now
            post_peak_stock = stock_pct
            post_peak_bench = bench_pct
            if post_hard_dt is None:
                post_soft_found = False
                post_soft_dt = None
                post_soft_val = np.nan
                post_soft_stock = np.nan
                post_soft_bench = np.nan

        t = (dt_now.hour, dt_now.minute)
        if not in_range(t, POST_FROM, POST_TO):
            return

        if post_hard_dt is None and cur_abs <= norm_thr:
            post_hard_dt = dt_now
            post_hard_val = float(dev_now)
            post_hard_stock = stock_pct
            post_hard_bench = bench_pct

        if post_hard_dt is None and isinstance(post_peak_dt, datetime) and post_peak_abs > 0:
            if dt_now >= post_peak_dt and cur_abs <= (float(post_peak_abs) / float(soft_ratio)):
                if (not post_soft_found) or (dt_now < post_soft_dt):
                    post_soft_found = True
                    post_soft_dt = dt_now
                    post_soft_val = float(dev_now)
                    post_soft_stock = stock_pct
                    post_soft_bench = bench_pct

    def finalize_post_event(reason="window_end"):
        nonlocal post_active, post_id
        if not post_active:
            return

        if post_hard_dt is not None:
            status = "hard"
            end_dt = post_hard_dt
            add_hard_delay("post", post_start_dt, post_hard_dt)
        elif post_soft_found and post_soft_dt is not None:
            status = "soft"
            end_dt = post_soft_dt
        else:
            status = "none"
            end_dt = None

        sk = post_sign_key()

        sb = floor_to_band(post_start_dt, start_band_minutes)
        if sb:
            start_bands_post_total[sb] += 1
            if status != "none":
                start_bands_post_any[sb] += 1
            if status == "hard":
                start_bands_post_hard[sb] += 1
            if status == "soft":
                start_bands_post_soft[sb] += 1

            update_timeband_by_class_sign(timebands_post_by_class_sign, "post", "start", sk, sb, status)

        counts_post["post"][status] += 1

        update_sigma_bins(sigma_bins_post, "post", sk, post_peak_abs, status)
        update_bench_bins(bench_bins_post, "post", "start", sk, post_start_bench, status)
        update_bench_bins(bench_bins_post, "post", "peak",  sk, post_peak_bench, status)

        if status != "none" and isinstance(end_dt, datetime):
            b = floor_to_band(end_dt, norm_band_minutes)
            if b:
                norm_bands_post_any[b] += 1
                if status == "hard":
                    norm_bands_post_hard[b] += 1
                if status == "soft":
                    norm_bands_post_soft[b] += 1

                update_timeband_by_class_sign(timebands_post_by_class_sign, "post", "norm", sk, b, status)
                update_bench_bins(bench_bins_post, "post", "norm", sk, (post_hard_bench if status == "hard" else post_soft_bench), status)

            push_last3_example(
                "post", sk, status,
                post_start_dt, end_dt,
                post_start_dev, (post_hard_val if status == "hard" else post_soft_val),
                post_peak_signed,
                post_start_stock, (post_hard_stock if status == "hard" else post_soft_stock),
                post_start_bench, (post_hard_bench if status == "hard" else post_soft_bench),
                start_time=post_start_dt.strftime("%H:%M") if isinstance(post_start_dt, datetime) else None,
                end_time=end_dt.strftime("%H:%M") if isinstance(end_dt, datetime) else None,
            )

        if include_events_post:
            post_events_buf.append({
                "post_id": int(post_id),
                "reason_finalized": reason,
                "start": {"dt": _dt_iso(post_start_dt), "dev": _json_safe(post_start_dev), "sign": int(post_start_sign)},
                "peak":  {"dt": _dt_iso(post_peak_dt), "abs": _json_safe(post_peak_abs), "signed": _json_safe(post_peak_signed)},
                "status": status,
            })

        post_id += 1
        reset_post_event()

    # ---------------- day boundary helpers ----------------
    def on_new_day():
        if pre_active:
            finalize_pre_event(reason="day_boundary")
        if intra_active:
            finalize_intra_event(reason="day_boundary")
        if post_active:
            finalize_post_event(reason="day_boundary")

    # ---------------- dictify helpers ----------------
    def dictify_sigma_bins(m):
        return {
            "pos": {b: dict(c) for b, c in m["pos"].items()},
            "neg": {b: dict(c) for b, c in m["neg"].items()},
        }

    def dictify_bench_bins(m):
        out = {}
        for which in ("start", "peak", "norm"):
            out[which] = {
                "pos": {b: dict(c) for b, c in m[which]["pos"].items()},
                "neg": {b: dict(c) for b, c in m[which]["neg"].items()},
            }
        return out

    def dictify_timebands_by_class_sign(m):
        out = {}
        for cls, blk in m.items():
            out[cls] = {}
            for which in ("start", "norm"):
                out[cls][which] = {
                    "pos": {band: dict(c) for band, c in blk[which]["pos"].items()},
                    "neg": {band: dict(c) for band, c in blk[which]["neg"].items()},
                }
        return out

    def dictify_last3(last3_map):
        out = {}
        for cls, by_sign in last3_map.items():
            out[cls] = {"pos": list(by_sign["pos"]), "neg": list(by_sign["neg"])}
        return out

    # ---------------- flush ticker (write files) ----------------
    def flush_current_ticker():
        nonlocal cur_ticker, cur_day
        nonlocal bench_name_seen, corr_static, beta_static, sigma_static

        if cur_ticker is None:
            return

        if pre_active:
            finalize_pre_event(reason="ticker_end")
        if intra_active:
            finalize_intra_event(reason="ticker_end")
        if post_active:
            finalize_post_event(reason="ticker_end")

        blue_r = class_rates(counts_pre["blue"])
        ark_r = class_rates(counts_pre["ark"])
        pr_r  = class_rates(counts_pre["print"])
        op_r  = class_rates(counts_pre["open"])
        gl_r  = class_rates(counts_pre["global"])
        intra_r = class_rates(counts_intra["intra"])
        post_r  = class_rates(counts_post["post"])

        events_pre_total = int(gl_r["total"])  # pre event count aligns with global counter
        events_intra_total = int(intra_r["total"])
        events_post_total  = int(post_r["total"])
        events_total = events_pre_total + events_intra_total + events_post_total

        # ✅ global filter for ALL outputs
        if events_total < int(min_events_per_ticker):
            reset_ticker_state()
            return

        # last10
        last10_print_days = []
        last10_peak_days = []
        for d in list(recent_days):
            snap = recent_by_day.get(d)
            if not snap:
                continue
            if snap.get("print") is not None:
                last10_print_days.append(snap["print"])
            if snap.get("peak") is not None:
                last10_peak_days.append(snap["peak"])

        pos_vals = list(last5_print_days_pos)
        neg_vals = list(last5_print_days_neg)

        # open series last10
        open_series_last10 = []
        for d in list(recent_days):
            series_map = open_series_by_day.get(d)
            if not series_map:
                continue
            pts = sorted(series_map.items(), key=lambda kv: kv[0])
            open_series_last10.append({
                "date": d,
                "points": [[dt_iso, _json_safe(val)] for dt_iso, val in pts],
            })

        payload = {
            "ticker": cur_ticker,
            "bench": bench_name_seen,
            "static": {"corr": _json_safe(corr_static), "beta": _json_safe(beta_static), "sigma": _json_safe(sigma_static)},
            "params": {
                "dev_thr": float(dev_thr), "norm_thr": float(norm_thr), "soft_ratio": float(soft_ratio),
                "windows": {
                    "blue": "00:01-03:59 (trigger allowed; peak frozen to 03:59; hard/soft within BLUE)",
                    "fixation_window": "00:05-09:29 (ARK peak frozen; ARK hard/soft within)",
                    "ignored_gaps": ["03:58-04:05", "07:58-08:05"],
                    "frozen_peak_until": "09:29",
                    "print_first": "09:30-09:35 (first tick only)",
                    "open_scan": "09:31-09:40 (scan + open dev series)",
                    "intra": "10:00-12:00 (trigger+normalize within)",
                    "post": "16:01-19:59 (trigger+normalize within)",
                    "global_priority": GLOBAL_PRIORITY,
                },
                "bins": {
                    "sigma": {"min": sigma_bin_min, "max": sigma_bin_max, "step": sigma_bin_step},
                    "bench": {"min": bench_bin_min, "max": bench_bin_max, "step": bench_bin_step},
                },
                "time_bands": {"start_band_minutes": start_band_minutes, "norm_band_minutes": norm_band_minutes},
                "best_rules": best_rules,
                "min_events_per_ticker": int(min_events_per_ticker),
                "open_series_downsample_seconds": int(open_series_downsample_seconds),
            },
      "stats": {
                "events_total": int(events_total),
                "pre": {
                    "events_total": int(events_pre_total),
                    "blue": blue_r,
                    "ark": ark_r,
                    "print": pr_r,
                    "open": op_r,
                    "global": {
                        **gl_r,
                        "labels": dict(global_labels_counter),
                        "best_label": global_labels_counter.most_common(1)[0][0] if global_labels_counter else None,
                    },
                    "hard_delay_avg_sec": {
                        "blue": _json_safe(avg_hard_delay("blue")),
                        "ark": _json_safe(avg_hard_delay("ark")),
                        "print": _json_safe(avg_hard_delay("print")),
                        "open": _json_safe(avg_hard_delay("open")),
                        "global": _json_safe(avg_hard_delay("global")),
                    },
                    "global_mean_peak_abs_when_normalized": {
                        "pos": _json_safe((global_norm_peak_sum["pos"] / global_norm_peak_cnt["pos"]) if global_norm_peak_cnt["pos"] else None),
                        "neg": _json_safe((global_norm_peak_sum["neg"] / global_norm_peak_cnt["neg"]) if global_norm_peak_cnt["neg"] else None),
                    },
                },
                "intra": {
                    "events_total": int(events_intra_total),
                    "intra": intra_r,
                    "hard_delay_avg_sec": {"intra": _json_safe(avg_hard_delay("intra"))},
                },
                "post": {
                    "events_total": int(events_post_total),
                    "post": post_r,
                    "hard_delay_avg_sec": {"post": _json_safe(avg_hard_delay("post"))},
                },
            },
            "time_bands": {
                "pre": {
                    "start_total": dict(start_bands_pre_total),
                    "start_any": dict(start_bands_pre_any),
                    "start_hard": dict(start_bands_pre_hard),
                    "start_soft": dict(start_bands_pre_soft),
                    "norm_any": dict(norm_bands_pre_any),
                    "norm_hard": dict(norm_bands_pre_hard),
                    "norm_soft": dict(norm_bands_pre_soft),
                },
                "intra": {
                    "start_total": dict(start_bands_intra_total),
                    "start_any": dict(start_bands_intra_any),
                    "start_hard": dict(start_bands_intra_hard),
                    "start_soft": dict(start_bands_intra_soft),
                    "norm_any": dict(norm_bands_intra_any),
                    "norm_hard": dict(norm_bands_intra_hard),
                    "norm_soft": dict(norm_bands_intra_soft),
                },
                "post": {
                    "start_total": dict(start_bands_post_total),
                    "start_any": dict(start_bands_post_any),
                    "start_hard": dict(start_bands_post_hard),
                    "start_soft": dict(start_bands_post_soft),
                    "norm_any": dict(norm_bands_post_any),
                    "norm_hard": dict(norm_bands_post_hard),
                    "norm_soft": dict(norm_bands_post_soft),
                },
                "pre_by_class_sign": dictify_timebands_by_class_sign(timebands_pre_by_class_sign),
                "intra_by_class_sign": dictify_timebands_by_class_sign(timebands_intra_by_class_sign),
                "post_by_class_sign": dictify_timebands_by_class_sign(timebands_post_by_class_sign),
            },
            "recent": {
                "last10_days": list(recent_days),
                "last10_print": last10_print_days,
                "last10_pre_peak_sigma": last10_peak_days,
                "last10_open_dev_series": open_series_last10,
                "last5_print": {
                    "pos": {
                        "values": pos_vals,
                        "mean": _json_safe(float(np.mean(pos_vals)) if pos_vals else None),
                        "median": _json_safe(float(np.median(pos_vals)) if pos_vals else None),
                    },
                    "neg": {
                        "values": neg_vals,
                        "mean": _json_safe(float(np.mean(neg_vals)) if neg_vals else None),
                        "median": _json_safe(float(np.median(neg_vals)) if neg_vals else None),
                    },
                },
            },

            "examples_last3_normalized": dictify_last3(last3_examples),

            "bins": {
                "sigma": {
                    "pre": {
                        "blue": dictify_sigma_bins(sigma_bins_pre["blue"]),
                        "ark": dictify_sigma_bins(sigma_bins_pre["ark"]),
                        "print": dictify_sigma_bins(sigma_bins_pre["print"]),
                        "open": dictify_sigma_bins(sigma_bins_pre["open"]),
                        "global": dictify_sigma_bins(sigma_bins_pre["global"]),
                    },
                    "intra": {"intra": dictify_sigma_bins(sigma_bins_intra["intra"])},
                    "post": {"post": dictify_sigma_bins(sigma_bins_post["post"])},
                },
                "bench": {
                    "pre": {
                        "blue": dictify_bench_bins(bench_bins_pre["blue"]),
                        "ark": dictify_bench_bins(bench_bins_pre["ark"]),
                        "print": dictify_bench_bins(bench_bins_pre["print"]),
                        "open": dictify_bench_bins(bench_bins_pre["open"]),
                        "global": dictify_bench_bins(bench_bins_pre["global"]),
                    },
                    "intra": {"intra": dictify_bench_bins(bench_bins_intra["intra"])},
                    "post": {"post": dictify_bench_bins(bench_bins_post["post"])},
                },
            },
        }

        if include_events_pre:
            payload["events_pre"] = list(pre_events_buf)
        if include_events_intra:
            payload["events_intra"] = list(intra_events_buf)
        if include_events_post:
            payload["events_post"] = list(post_events_buf)

        onefile_f.write(json.dumps(payload, ensure_ascii=False) + "\n")

        # SUMMARY row
        row = {
            "ticker": cur_ticker,
            "bench": bench_name_seen,
            "events_total": int(events_total),
            "events_pre_total": int(events_pre_total),
            "events_intra_total": int(events_intra_total),
            "events_post_total": int(events_post_total),

            "blue_any_rate": _json_safe(blue_r["rate_any"]),
            "blue_hard_rate": _json_safe(blue_r["rate_hard"]),
            "blue_soft_rate": _json_safe(blue_r["rate_soft"]),

            "ark_any_rate": _json_safe(ark_r["rate_any"]),
            "ark_hard_rate": _json_safe(ark_r["rate_hard"]),
            "ark_soft_rate": _json_safe(ark_r["rate_soft"]),

            "print_any_rate": _json_safe(pr_r["rate_any"]),
            "print_hard_rate": _json_safe(pr_r["rate_hard"]),
            "print_soft_rate": _json_safe(pr_r["rate_soft"]),

            "open_any_rate": _json_safe(op_r["rate_any"]),
            "open_hard_rate": _json_safe(op_r["rate_hard"]),
            "open_soft_rate": _json_safe(op_r["rate_soft"]),

            "global_any_rate": _json_safe(gl_r["rate_any"]),
            "global_hard_rate": _json_safe(gl_r["rate_hard"]),
            "global_soft_rate": _json_safe(gl_r["rate_soft"]),

            "intra_any_rate": _json_safe(intra_r["rate_any"]),
            "intra_hard_rate": _json_safe(intra_r["rate_hard"]),
            "intra_soft_rate": _json_safe(intra_r["rate_soft"]),

            "post_any_rate": _json_safe(post_r["rate_any"]),
            "post_hard_rate": _json_safe(post_r["rate_hard"]),
            "post_soft_rate": _json_safe(post_r["rate_soft"]),

            "corr": _json_safe(corr_static),
            "beta": _json_safe(beta_static),
            "sigma": _json_safe(sigma_static),
        }
        pd.DataFrame([row], columns=summary_cols).to_csv(output_summary_csv, mode="a", header=False, index=False)

        # BEST PARAMS: keep + ADD best_windows_any for ALL classes
        def median_or_none(arr):
            arr = list(arr)
            return _json_safe(float(np.median(arr)) if arr else None)

        best_windows_any = {
            "sigma_peak_bins": {
                "blue":  {"pos": stitch_numeric_bin_intervals_from_any(sigma_bins_pre["blue"]["pos"], step=sigma_bin_step),
                          "neg": stitch_numeric_bin_intervals_from_any(sigma_bins_pre["blue"]["neg"], step=sigma_bin_step)},
                "ark":   {"pos": stitch_numeric_bin_intervals_from_any(sigma_bins_pre["ark"]["pos"], step=sigma_bin_step),
                          "neg": stitch_numeric_bin_intervals_from_any(sigma_bins_pre["ark"]["neg"], step=sigma_bin_step)},
                "print": {"pos": stitch_numeric_bin_intervals_from_any(sigma_bins_pre["print"]["pos"], step=sigma_bin_step),
                          "neg": stitch_numeric_bin_intervals_from_any(sigma_bins_pre["print"]["neg"], step=sigma_bin_step)},
                "open":  {"pos": stitch_numeric_bin_intervals_from_any(sigma_bins_pre["open"]["pos"], step=sigma_bin_step),
                          "neg": stitch_numeric_bin_intervals_from_any(sigma_bins_pre["open"]["neg"], step=sigma_bin_step)},
                "global":{"pos": stitch_numeric_bin_intervals_from_any(sigma_bins_pre["global"]["pos"], step=sigma_bin_step),
                          "neg": stitch_numeric_bin_intervals_from_any(sigma_bins_pre["global"]["neg"], step=sigma_bin_step)},
                "intra": {"pos": stitch_numeric_bin_intervals_from_any(sigma_bins_intra["intra"]["pos"], step=sigma_bin_step),
                          "neg": stitch_numeric_bin_intervals_from_any(sigma_bins_intra["intra"]["neg"], step=sigma_bin_step)},
                "post":  {"pos": stitch_numeric_bin_intervals_from_any(sigma_bins_post["post"]["pos"], step=sigma_bin_step),
                          "neg": stitch_numeric_bin_intervals_from_any(sigma_bins_post["post"]["neg"], step=sigma_bin_step)},
            },
            "bench_peak_bins": {
                "blue":  {"pos": stitch_numeric_bin_intervals_from_any(bench_bins_pre["blue"]["peak"]["pos"], step=bench_bin_step),
                          "neg": stitch_numeric_bin_intervals_from_any(bench_bins_pre["blue"]["peak"]["neg"], step=bench_bin_step)},
                "ark":   {"pos": stitch_numeric_bin_intervals_from_any(bench_bins_pre["ark"]["peak"]["pos"], step=bench_bin_step),
                          "neg": stitch_numeric_bin_intervals_from_any(bench_bins_pre["ark"]["peak"]["neg"], step=bench_bin_step)},
                "print": {"pos": stitch_numeric_bin_intervals_from_any(bench_bins_pre["print"]["peak"]["pos"], step=bench_bin_step),
                          "neg": stitch_numeric_bin_intervals_from_any(bench_bins_pre["print"]["peak"]["neg"], step=bench_bin_step)},
                "open":  {"pos": stitch_numeric_bin_intervals_from_any(bench_bins_pre["open"]["peak"]["pos"], step=bench_bin_step),
                          "neg": stitch_numeric_bin_intervals_from_any(bench_bins_pre["open"]["peak"]["neg"], step=bench_bin_step)},
                "global":{"pos": stitch_numeric_bin_intervals_from_any(bench_bins_pre["global"]["peak"]["pos"], step=bench_bin_step),
                          "neg": stitch_numeric_bin_intervals_from_any(bench_bins_pre["global"]["peak"]["neg"], step=bench_bin_step)},
                "intra": {"pos": stitch_numeric_bin_intervals_from_any(bench_bins_intra["intra"]["peak"]["pos"], step=bench_bin_step),
                          "neg": stitch_numeric_bin_intervals_from_any(bench_bins_intra["intra"]["peak"]["neg"], step=bench_bin_step)},
                "post":  {"pos": stitch_numeric_bin_intervals_from_any(bench_bins_post["post"]["peak"]["pos"], step=bench_bin_step),
                          "neg": stitch_numeric_bin_intervals_from_any(bench_bins_post["post"]["peak"]["neg"], step=bench_bin_step)},
            },
            "time_start_bands": {
                "blue": {
                    "pos": stitch_timeband_intervals_from_any(timebands_pre_by_class_sign["blue"]["start"]["pos"]),
                    "neg": stitch_timeband_intervals_from_any(timebands_pre_by_class_sign["blue"]["start"]["neg"]),
                },
                "ark": {
                    "pos": stitch_timeband_intervals_from_any(timebands_pre_by_class_sign["ark"]["start"]["pos"]),
                    "neg": stitch_timeband_intervals_from_any(timebands_pre_by_class_sign["ark"]["start"]["neg"]),
                },
                "print": {
                    "pos": stitch_timeband_intervals_from_any(timebands_pre_by_class_sign["print"]["start"]["pos"]),
                    "neg": stitch_timeband_intervals_from_any(timebands_pre_by_class_sign["print"]["start"]["neg"]),
                },
                "open": {
                    "pos": stitch_timeband_intervals_from_any(timebands_pre_by_class_sign["open"]["start"]["pos"]),
                    "neg": stitch_timeband_intervals_from_any(timebands_pre_by_class_sign["open"]["start"]["neg"]),
                },
                "global": {
                    "pos": stitch_timeband_intervals_from_any(timebands_pre_by_class_sign["global"]["start"]["pos"]),
                    "neg": stitch_timeband_intervals_from_any(timebands_pre_by_class_sign["global"]["start"]["neg"]),
                },
                "intra": {
                    "pos": stitch_timeband_intervals_from_any(timebands_intra_by_class_sign["intra"]["start"]["pos"]),
                    "neg": stitch_timeband_intervals_from_any(timebands_intra_by_class_sign["intra"]["start"]["neg"]),
                },
                "post": {
                    "pos": stitch_timeband_intervals_from_any(timebands_post_by_class_sign["post"]["start"]["pos"]),
                    "neg": stitch_timeband_intervals_from_any(timebands_post_by_class_sign["post"]["start"]["neg"]),
                },
            }
        }

        best = {
            "ticker": cur_ticker,
            "bench": bench_name_seen,
            "static": {"corr": _json_safe(corr_static), "beta": _json_safe(beta_static), "sigma": _json_safe(sigma_static)},

            "dev_print_last5_median": {
                "pos": median_or_none(last5_print_days_pos),
                "neg": median_or_none(last5_print_days_neg),
            },

            "totals": {
                "events_total": int(events_total),
                "pre_total": int(events_pre_total),
                "intra_total": int(events_intra_total),
                "post_total": int(events_post_total),
            },

            "ratings": {
                "blue": _json_safe(blue_r["rate_any"]),
                "ark": _json_safe(ark_r["rate_any"]),
                "print": _json_safe(pr_r["rate_any"]),
                "open": _json_safe(op_r["rate_any"]),
                "intra": _json_safe(intra_r["rate_any"]),
                "post": _json_safe(post_r["rate_any"]),
                "global": _json_safe(gl_r["rate_any"]),
            },

            "hard_soft_share": {
                "blue":  {"hard": int(blue_r["hard"]), "soft": int(blue_r["soft"]), "hard_share": _json_safe(blue_r["hard_share_in_norm"])},
                "ark":   {"hard": int(ark_r["hard"]),  "soft": int(ark_r["soft"]),  "hard_share": _json_safe(ark_r["hard_share_in_norm"])},
                "print": {"hard": int(pr_r["hard"]),   "soft": int(pr_r["soft"]),   "hard_share": _json_safe(pr_r["hard_share_in_norm"])},
                "open":  {"hard": int(op_r["hard"]),   "soft": int(op_r["soft"]),   "hard_share": _json_safe(op_r["hard_share_in_norm"])},
                "intra": {"hard": int(intra_r["hard"]), "soft": int(intra_r["soft"]), "hard_share": _json_safe(intra_r["hard_share_in_norm"])},
                "post":  {"hard": int(post_r["hard"]),  "soft": int(post_r["soft"]),  "hard_share": _json_safe(post_r["hard_share_in_norm"])},
                "global":{"hard": int(gl_r["hard"]),   "soft": int(gl_r["soft"]),   "hard_share": _json_safe(gl_r["hard_share_in_norm"])},
            },

            "avg_hard_delay_sec": {
                "blue": _json_safe(avg_hard_delay("blue")),
                "ark": _json_safe(avg_hard_delay("ark")),
                "print": _json_safe(avg_hard_delay("print")),
                "open": _json_safe(avg_hard_delay("open")),
                "intra": _json_safe(avg_hard_delay("intra")),
                "post": _json_safe(avg_hard_delay("post")),
                "global": _json_safe(avg_hard_delay("global")),
            },

            "best_windows_any": {
                "rule": {"min_rate": 0.60, "min_total": 4, "rate": "(hard+soft)/total"},
                "stitched": best_windows_any,
            },

            "params": {"dev_thr": float(dev_thr), "norm_thr": float(norm_thr), "soft_ratio": float(soft_ratio)},
        }

        best_params_f.write(json.dumps(best, ensure_ascii=False) + "\n")
        reset_ticker_state()
        
    # ---------------- processing chunks ----------------
    def process_chunk(chunk: "pd.DataFrame", ci: int):
        nonlocal cur_ticker, cur_day
        nonlocal bench_name_seen, static_triplet_set, corr_static, beta_static, sigma_static
        nonlocal pre_active, intra_active, post_active

        req = {"ticker", "date", "dt", "dev_sig"}
        if not req.issubset(chunk.columns):
            raise KeyError(f"Input must contain columns: {sorted(req)}")

        # If not sorted — make it sorted once (vectorized), no per-row parse_dt
        if not assume_sorted:
            chunk["dt"] = pd.to_datetime(chunk["dt"], errors="coerce", utc=True)
            chunk.sort_values(["ticker", "date", "dt"], inplace=True)

        def col(name):
            return chunk[name] if name in chunk.columns else pd.Series(np.nan, index=chunk.index)

        s_ticker = col("ticker")
        s_date   = col("date")

        # ✅ Vectorized datetime + numeric parse
        s_dt_ts  = pd.to_datetime(col("dt"), errors="coerce", utc=True)  # tz-aware Timestamp (UTC)
        s_dev    = pd.to_numeric(col("dev_sig"), errors="coerce")

        s_bench_name = col("bench")
        s_corr  = col("corr")
        s_beta  = col("beta")
        s_sigma = col("sigma")

        s_bench_num = (
            pd.to_numeric(col(BENCH_NUM_FIELD), errors="coerce")
            if BENCH_NUM_FIELD in chunk.columns
            else pd.Series(np.nan, index=chunk.index)
        )
        s_stock_pct = (
            pd.to_numeric(col(STOCK_NUM_FIELD), errors="coerce")
            if STOCK_NUM_FIELD in chunk.columns
            else pd.Series(np.nan, index=chunk.index)
        )

        # ✅ Build a fast mask upfront (drop bad dt/dev and ignored windows) BEFORE loop
        dt_ok  = s_dt_ts.notna()
        dev_ok = np.isfinite(s_dev.to_numpy(dtype="float64", copy=False))
        mask = (dt_ok.to_numpy(copy=False) & dev_ok)

        if mask.any():
            # ignored windows mask using hour/min (vectorized)
            dt2 = s_dt_ts[mask]
            h = dt2.dt.hour.to_numpy(dtype="int16", copy=False)
            m = dt2.dt.minute.to_numpy(dtype="int16", copy=False)

            # vectorized ignored windows:
            # IGNORE_WINDOWS = [((3, 58), (4, 5)), ((7, 58), (8, 5))]
            # condition: a <= (h,m) <= b for any window
            def _in_win(h, m, a, b):
                ah, am = a
                bh, bm = b
                return ((h > ah) | ((h == ah) & (m >= am))) & ((h < bh) | ((h == bh) & (m <= bm)))

            ig = np.zeros_like(h, dtype=bool)
            for a, b in IGNORE_WINDOWS:
                ig |= _in_win(h, m, a, b)

            # apply back to global mask
            idx_mask = np.flatnonzero(mask)
            mask[idx_mask] = ~ig

        # nothing useful in chunk
        if not mask.any():
            return

        # ✅ set bench_name_seen once per chunk (first non-null)
        if bench_name_seen is None and "bench" in chunk.columns:
            bn = s_bench_name[mask]
            if len(bn) > 0:
                first_bn = bn.dropna()
                if len(first_bn) > 0:
                    bench_name_seen = first_bn.iloc[0]

        # ✅ set static triplet once per chunk (first row with corr/beta/sigma present)
        if not static_triplet_set and all(x in chunk.columns for x in ("corr", "beta", "sigma")):
            cc = s_corr[mask]
            bb = s_beta[mask]
            ss = s_sigma[mask]
            ok = (cc.notna() & bb.notna() & ss.notna())
            if ok.any():
                j = ok.idxmax()  # first True index
                corr_static, beta_static, sigma_static = cc.loc[j], bb.loc[j], ss.loc[j]
                static_triplet_set = True

        # ✅ Build compact arrays (keep original order)
        # Convert to python datetime ONLY for the filtered rows (cheap vs per-row parse_dt)
        dt_py = s_dt_ts[mask].dt.to_pydatetime()  # ndarray of datetime (tz-aware)
        tk_arr = s_ticker[mask].to_numpy(copy=False)
        ds_arr = s_date[mask].to_numpy(copy=False)
        dev_arr = s_dev[mask].to_numpy(dtype="float64", copy=False)
        bench_arr = s_bench_num[mask].to_numpy(dtype="float64", copy=False)
        stock_arr = s_stock_pct[mask].to_numpy(dtype="float64", copy=False)

        # Precompute hour/min for fast tuple t
        dt2 = s_dt_ts[mask]
        h_arr = dt2.dt.hour.to_numpy(dtype="int16", copy=False)
        m_arr = dt2.dt.minute.to_numpy(dtype="int16", copy=False)

        n = len(dev_arr)
        for i in range(n):
            tk = tk_arr[i]
            ds = ds_arr[i]
            dt_now = dt_py[i]         # datetime with tzinfo
            dev_now = dev_arr[i]      # float
            bench_num = bench_arr[i]  # float (may be nan)
            stock_pct = stock_arr[i]  # float (may be nan)

            # cheap time tuple
            t = (int(h_arr[i]), int(m_arr[i]))

            # ticker boundary
            if cur_ticker is not None and tk != cur_ticker:
                flush_current_ticker()
                cur_ticker, cur_day = tk, ds
                on_new_day()

            if cur_ticker is None:
                cur_ticker, cur_day = tk, ds
                on_new_day()

            # day boundary
            if ds != cur_day:
                cur_day = ds
                on_new_day()

            # finalize PRE after OPEN window
            if pre_active and (t > OPEN_TO):
                finalize_pre_event(reason="passed_open_window")

            # start PRE in BLUE or ARK window
            if (not pre_active) and (in_range(t, BLUE_FROM, BLUE_TO) or in_range(t, ARK_FROM, ARK_TO)):
                if abs(float(dev_now)) >= dev_thr:
                    start_pre_event(dt_now, dev_now, stock_pct, bench_num)

            if pre_active:
                pre_process_tick(dt_now, dev_now, stock_pct, bench_num)

            # finalize INTRA after window
            if intra_active and (t > INTRA_TO):
                finalize_intra_event(reason="passed_intra_window")

            # start INTRA inside 10:00-12:00
            if (not intra_active) and in_range(t, INTRA_FROM, INTRA_TO):
                if abs(float(dev_now)) >= dev_thr:
                    start_intra_event(dt_now, dev_now, stock_pct, bench_num)

            if intra_active and in_range(t, INTRA_FROM, INTRA_TO):
                intra_process_tick(dt_now, dev_now, stock_pct, bench_num)

            # finalize POST after window
            if post_active and (t > POST_TO):
                finalize_post_event(reason="passed_post_window")

            # start POST inside 16:01-19:59
            if (not post_active) and in_range(t, POST_FROM, POST_TO):
                if abs(float(dev_now)) >= dev_thr:
                    start_post_event(dt_now, dev_now, stock_pct, bench_num)

            if post_active and in_range(t, POST_FROM, POST_TO):
                post_process_tick(dt_now, dev_now, stock_pct, bench_num)


    # ---------------- main read loop ----------------
    t0 = time.time()
    total_rows = 0
    last_rows = 0
    last_ts = t0

    is_parquet = str(input_path).lower().endswith((".parquet", ".pq", ".parq"))
    print(
        f"▶️ START v12 exporter+BLUE+POST file={input_path} parquet={is_parquet} "
        f"dev_thr={dev_thr} norm_thr={norm_thr} soft_ratio={soft_ratio} min_events={min_events_per_ticker}"
    )

    try:
        if is_parquet and parquet_use_pyarrow:
            import pyarrow.parquet as pq
            pf = pq.ParquetFile(input_path)

            wanted = ["ticker", "date", "dt", "dev_sig", "bench", "corr", "beta", "sigma", STOCK_NUM_FIELD, BENCH_NUM_FIELD]
            cols = [c for c in wanted if c in pf.schema.names]

            for ci in range(pf.num_row_groups):
                chunk = pf.read_row_group(ci, columns=cols).to_pandas()
                process_chunk(chunk, ci + 1)
                total_rows += len(chunk)

                if (ci + 1) % log_every_n_chunks == 0:
                    now = time.time()
                    rps = (total_rows - last_rows) / max(now - last_ts, 1e-6)
                    print(f"[rg {ci+1:>4}/{pf.num_row_groups}] rows={total_rows:,} speed={rps:,.0f}/s elapsed={now-t0:,.1f}s")
                    last_rows, last_ts = total_rows, now

                # cheaper cleanup (avoid gc.collect each chunk)
                del chunk
                if (ci + 1) % max(10, log_every_n_chunks * 2) == 0:
                    gc.collect()

        elif not is_parquet:
            reader = pd.read_csv(input_path, compression="infer", low_memory=False, chunksize=csv_chunksize)
            for ci, chunk in enumerate(reader, 1):
                process_chunk(chunk, ci)
                total_rows += len(chunk)

                if ci % log_every_n_chunks == 0:
                    now = time.time()
                    rps =qar = (total_rows - last_rows) / max(now - last_ts, 1e-6)
                    print(f"[chunk {ci:>5}] rows={total_rows:,} speed={rps:,.0f}/s elapsed={now-t0:,.1f}s")
                    last_rows, last_ts = total_rows, now

                del chunk
                if ci % max(10, log_every_n_chunks * 2) == 0:
                    gc.collect()

        else:
            df = pd.read_parquet(input_path)
            step = 1_000_000
            for ci, start in enumerate(range(0, len(df), step), 1):
                chunk = df.iloc[start:start + step]
                process_chunk(chunk, ci)
                total_rows += len(chunk)

                if ci % log_every_n_chunks == 0:
                    now = time.time()
                    rps = (total_rows - last_rows) / max(now - last_ts, 1e-6)
                    print(f"[chunk {ci:>5}] rows={total_rows:,} speed={rps:,.0f}/s elapsed={now-t0:,.1f}s")
                    last_rows, last_ts = total_rows, now

                del chunk
                if ci % max(10, log_every_n_chunks * 2) == 0:
                    gc.collect()

        flush_current_ticker()
        print(f"🏁 DONE rows={total_rows:,} -> onefile={output_onefile_jsonl} summary={output_summary_csv} best_params={output_best_params_jsonl}")

    finally:
        onefile_f.close()
        best_params_f.close()



In [5]:
# devsig_stream_stats_v12_exporter(
#     input_path="ARBITRAGE/final_filtered.parquet",
#     output_onefile_jsonl="ARBITRAGE/onefile.jsonl",
#     output_summary_csv="ARBITRAGE/summary.csv",
#     output_best_params_jsonl="ARBITRAGE/best_params.jsonl",

#     dev_thr=0.30,
#     norm_thr=0.10,
#     soft_ratio=3.0,

#     include_events_pre=False,
#     include_events_intra=False,
#     max_events_per_ticker=500,

#     min_events_per_ticker=10,

#     start_band_minutes=30,
#     norm_band_minutes=30,

#     sigma_bin_min=0.2,
#     sigma_bin_max=2.7,
#     sigma_bin_step=0.1,

#     bench_bin_min=-3.0,
#     bench_bin_max=3.0,
#     bench_bin_step=0.1,

#     open_series_downsample_seconds=60,
# )


In [6]:
from pathlib import Path
import os

def _resolve_orion_paths(strategy_code: str):
    """
    Strategy reads FINAL from:
      1) env FINAL_PARQUET_PATH (preferred; runner sets it)
      2) ORION_HOME/CRACEN/final.parquet
      3) auto-find OriON by walking up from cwd
    Writes outputs to:
      ORION_HOME/signals/{strategy_code}/...
    """
    # 1) preferred: runner sets FINAL_PARQUET_PATH + SIGNALS_DIR
    final_env = os.environ.get("FINAL_PARQUET_PATH")
    sig_env   = os.environ.get("SIGNALS_DIR")
    orion_env = os.environ.get("ORION_HOME")

    if final_env:
        final_path = Path(final_env).expanduser().resolve()
    else:
        final_path = None

    if sig_env:
        signals_base = Path(sig_env).expanduser().resolve()
    else:
        signals_base = None

    # 2) fallback: ORION_HOME
    if (final_path is None or signals_base is None) and orion_env:
        orion_home = Path(orion_env).expanduser().resolve()
        if final_path is None:
            final_path = (orion_home / "CRACEN" / "final.parquet").resolve()
        if signals_base is None:
            signals_base = (orion_home / "signals").resolve()

    # 3) fallback: search upward for OriON folder
    if final_path is None or signals_base is None:
        here = Path.cwd().resolve()
        orion_home = None
        for parent in [here] + list(here.parents):
            if parent.name.lower() == "orion":
                orion_home = parent
                break
            cand = parent / "OriON"
            if cand.exists() and cand.is_dir():
                orion_home = cand.resolve()
                break
        if orion_home is None:
            raise RuntimeError("Cannot locate OriON. Set ORION_HOME (recommended).")
        if final_path is None:
            final_path = (orion_home / "CRACEN" / "final.parquet").resolve()
        if signals_base is None:
            signals_base = (orion_home / "signals").resolve()

    out_dir = (signals_base / strategy_code.lower()).resolve()
    out_dir.mkdir(parents=True, exist_ok=True)

    if not final_path.exists():
        raise FileNotFoundError(f"FINAL parquet not found: {final_path}")

    return final_path, out_dir


FINAL_PATH, OUT_DIR = _resolve_orion_paths("arbitrage")

devsig_stream_stats_v12_exporter(
    input_path=str(FINAL_PATH),

    # Запис у стиснені файли (.gz)
    output_onefile_jsonl=str(OUT_DIR / "onefile.jsonl.gz"),
    output_best_params_jsonl=str(OUT_DIR / "best_params.jsonl.gz"),

    # summary залишається незжатим
    output_summary_csv=str(OUT_DIR / "summary.csv"),

    dev_thr=0.30,
    norm_thr=0.10,
    soft_ratio=3.0,

    include_events_pre=False,
    include_events_intra=False,
    max_events_per_ticker=500,

    min_events_per_ticker=10,

    start_band_minutes=30,
    norm_band_minutes=30,

    sigma_bin_min=0.2,
    sigma_bin_max=2.7,
    sigma_bin_step=0.1,

    bench_bin_min=-3.0,
    bench_bin_max=3.0,
    bench_bin_step=0.1,

    open_series_downsample_seconds=60,
)


▶️ START v12 exporter+BLUE+POST file=C:\datum-api-examples-main\OriON\CRACEN\final.parquet parquet=True dev_thr=0.3 norm_thr=0.1 soft_ratio=3.0 min_events=10


[rg    5/6648] rows=117,965 speed=335,128/s elapsed=0.4s
[rg   10/6648] rows=199,930 speed=470,516/s elapsed=0.5s


[rg   15/6648] rows=336,692 speed=522,429/s elapsed=0.8s
[rg   20/6648] rows=397,330 speed=430,081/s elapsed=0.9s


[rg   25/6648] rows=523,997 speed=511,320/s elapsed=1.2s
[rg   30/6648] rows=610,011 speed=540,635/s elapsed=1.3s


[rg   35/6648] rows=679,450 speed=444,672/s elapsed=1.5s
[rg   40/6648] rows=770,108 speed=551,483/s elapsed=1.7s


...

[rg 6605/6648] rows=104,438,841 speed=493,416/s elapsed=208.4s
[rg 6610/6648] rows=104,499,152 speed=481,302/s elapsed=208.5s


[rg 6615/6648] rows=104,582,815 speed=498,966/s elapsed=208.7s
[rg 6620/6648] rows=104,679,619 speed=541,001/s elapsed=208.8s


[rg 6625/6648] rows=104,717,953 speed=424,312/s elapsed=208.9s
[rg 6630/6648] rows=104,814,845 speed=479,455/s elapsed=209.1s


[rg 6635/6648] rows=104,933,208 speed=509,365/s elapsed=209.4s
[rg 6640/6648] rows=105,007,157 speed=494,001/s elapsed=209.5s


[rg 6645/6648] rows=105,066,459 speed=446,294/s elapsed=209.7s
🏁 DONE rows=105,113,179 -> onefile=C:\datum-api-examples-main\OriON\signals\arbitrage\onefile.jsonl.gz summary=C:\datum-api-examples-main\OriON\signals\arbitrage\summary.csv best_params=C:\datum-api-examples-main\OriON\signals\arbitrage\best_params.jsonl.gz
